<a href="https://colab.research.google.com/github/Bob-Harper/desktop-tutorial/blob/main/Calliope_Prompt_Parrot_v2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Calliope's Archivarium! 
(originally prompt parrot, but now customized for Artistic Misfits)

This notebook is designed to train language model on a list of your prompts,generate prompts in your style, and synthesize wonderful surreal images! ✨ 

# Instructions
This version of the notebook uses the Onenote Prompts from Lilyuni  Instructions are pared down for ease of use.  

### Option 1) Built-in Prompts

Run all cells! Default settings will use the builtin prompts by default, train GP2 and produce art!

No other instructions needed as built-in prompts have already been replaced by the OneNote Prompts.

Just go down the list, run each one in turn.

#0. Setup

In [10]:
#@title 0.1 GPU Check.  If it doesn't say "GPU 0: Tesla T4" close out and try again.

!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-61d55f60-4b43-b8fb-7bc5-9bac6501b327)


In [11]:
# @title Mount Drive - needs your Google Drive to store files as they are created.  leave as is unless you REALLY need to change them.

from google.colab import drive
from pathlib import Path
import os

ROOT_FOLDER = "generative" # @param {type:"string"}
PROJECT_FOLDER = "prompt_parrot" # @param {type:"string"}
COMMON_ASSETS_FOLDER = "parrot_commons" # @param {type:"string"}
MOUNTED_PATH = Path('/content/drive')
MYDRIVE_PATH = MOUNTED_PATH / "MyDrive"
PROJECT_PATH = MYDRIVE_PATH /  ROOT_FOLDER / PROJECT_FOLDER
COMMON_ASSETS_PATH = MYDRIVE_PATH /  ROOT_FOLDER / COMMON_ASSETS_FOLDER

CACHE_PATH = COMMON_ASSETS_PATH / "models"
IMAGES_OUT = PROJECT_PATH/ "images_out"

drive.mount(str(MOUNTED_PATH))

os.makedirs(PROJECT_PATH, exist_ok=True)
os.makedirs(CACHE_PATH, exist_ok=True)
os.makedirs(IMAGES_OUT, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#@title 0.3 Install Dependencies.  Installs stuff needed to make it go.
!pip install transformers datasets scipy ftfy git+https://github.com/kyrick/parrot-tools.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/kyrick/parrot-tools.git to /tmp/pip-req-build-7rrbs23s
  Running command git clone -q https://github.com/kyrick/parrot-tools.git /tmp/pip-req-build-7rrbs23s
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [13]:
#@title 0.4 Imports.  More stuff to make it work. Just keep clicking.

# from itertools import chain
# from typing import Optional
# import textwrap
import os
import textwrap
import random

from parrot_tools.generate import run_prompts, BatchSettings, Prompt, set_scheduler

# GPT2 training imports
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TextDataset, 
    DataCollatorForLanguageModeling,
    Trainer, 
    TrainingArguments
)
import datasets
from google.colab import files

# stable diffusion imports
import torch
from diffusers import StableDiffusionPipeline

# no need for caching with tiny datasets. More trouble than it's worth
datasets.disable_caching()

from google.colab import output
output.enable_custom_widget_manager()

from huggingface_hub import notebook_login

# THIS IS A DIRTY HACK TO SILENCE THE PROGRESS BAR
# THE TQDM PROGRESS BAR BOTTLENECKS CELL OUTPUT AND SLOWS THE NOTEBOOK
from tqdm.auto import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [14]:
#@title 0.5 Define Functions.  Even more "make it go" stuff.
start_token = "<BOP>"
pad_token = "<PAD>"
end_token = "<EOP>"
special_tokens_dict = {
    'bos_token': start_token, 
    'eos_token': end_token, 
    'pad_token': pad_token
}

def tokenize_line_by_line(tokenizer, prompts_dataset):
    def tokenize_function(dataset):
        # Remove empty lines

        dataset["prompts"] = [
            start_token + prompt.strip() + end_token 
            for prompt in dataset["prompts"] 
            if len(prompt.strip()) > 0]

        return tokenizer(
            dataset["prompts"],
            padding="max_length",
            truncation=True,
            # return_special_tokens_mask=True,
            max_length=tokenizer.model_max_length,
        )

    tokenized_dataset = prompts_dataset.map(
        tokenize_function,
        batched=True,
        batch_size=10000,
        drop_last_batch=False,
        remove_columns=["prompts"],
    ).map(remove_columns=['attention_mask'])
    return tokenized_dataset


def tokenize_group(tokenizer, prompts_dataset, shuffles=4):
    def group_texts(examples):
        block_size = tokenizer.model_max_length
        
        prompts = examples["prompts"]

        shuffled_prompts = ""
        for _ in range(shuffles):
            random.shuffle(prompts)
            concat =  "".join([start_token + prompt.strip() + " " + end_token for prompt in prompts if len(prompt.strip()) > 0])
            shuffled_prompts += concat

        input_ids = tokenizer(shuffled_prompts)["input_ids"]

        result = []
        for i in range(0, len(input_ids) - block_size + 1, block_size):  # Truncate in block of block_size
            result.append(
                tokenizer.build_inputs_with_special_tokens(input_ids[i : i + block_size])
            )
        return {"input_ids":result}

    tokenized_datasets = prompts_dataset.map(
        group_texts,
        batched=True,
        batch_size=10000,
        remove_columns=["prompts"],
    )
    return tokenized_datasets



In [ ]:
#@title 0.6 Built In Prompts - This is all we need.  We can add more prompts to this whenever we want, use the "show code" and be sure to ONLY replace the prompts, not the lines with punctuation or code.  IF YOU HAVE ALREADY RUN TRAINING AND HAVE A SAVED MODEL YOU CAN SKIP THIS IF YOU ARE NOT ADDING NEW PROMPTS.

builtin_prompts = """\
“Hotel Vanacy”, Intricate, wooden, mossy, Digital, Scenic, Hyperdetailed, Complex. Luis Royo, Seb, Deep Colors, foreboding greens, Bagshaw, Chevrier, Ferri, Kaluta, Victor Adame Minguez, Ruan Jia. Cinematic, WLOP, Homestead, fallen windows, fallen fences, A financier anthropomorphic weasel 
A wide shot photo of a detailed!!!!! jellyfish; fantasy, concept art, elegant, crisp, emisive lighting, bioluminescent, colorful, shimmering, prismatic; Steve McCurry, James Jean, artstation; 8k, unreal engine, octane, mystical light, evocative, ornate, award-winning, masterpiece, photorealistic 
the portrait of a cute dog , a highly intricate and hyperdetailed matte photography by Ismail Inceoglu, Sakiyama, James Gurney, Victo Ngai, M.W Kaluta and David Friedrich, album cover art, long exposure, cel-shaded, a masterpiece, ZBrushCentral, CGSociety, deep colors 
Portrait of a cute and adorable gold and blue cat. Ankha from Animal Crossing by Jean Baptiste monge, Ismail Inceoglu, Karol Bak, Tyler Edlin, wearing Pharoah's garb, album art, anthromorphic, character design, detailed painting, hyperdetailed, hyperrealism, photorealism, masterpiece, by Greg Rutkowski, trending on artstation 
Symmetry!! product render poster vivid colors divine proportion owl, ice and snow, glowing fog intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration 
a portrait of a psychedelic cat owl figure by naoto hattori, android jones and chris dyer, deep bold colors, galactic entity, depth of field, intricate beautiful painting, billions of details, octane render, portal, 8 k, detailed vector, trending on artstation, cgisociety, neural, runic, wow!!!!!!!!!! 
A mad Lion , a highly intricate and hyper-detailed painting by Ismail Inceoglu, Jean-Baptiste Monge, Jaroslaw Jasnikowski, fantasy art, hyperrealism, deep colors, photorealism 
baby dragon, scales, photorealistic, 8K, trending on CGSociety, painterly 
Sultry pose, kawaii western peasant girl, newborn chicks in hand, long hair, white tischel; LinaTesch, RossTran, JeanBaptisteMonge; vibrant colors, farm life, ink illustration, digital illustration, album cover art, masterpiece; 16k, hyperdetailed, hyperrealism, deep colors, vivid, octane 
"Goldilocks after she was frightened by the three bears"; grim, macabre, tough, gods-rays, earth tones, dark ethereal fantasy; Sunny Carvalho, Horka Dolls, Brian Kesinger, Beatrix Potter, Johan Egerkranz; character art, concept art, 16k, photorealism, unreal engine, octane, ink illustration, digital illustration, surrealism 
A portrait of a cute anthro deer , a highly intricate and hyperdetailed matte painting by Victo Ngai, Junji Ito, James Gurney, Ismail Inceoglu, Gregorio Catarino, Dan Mumford and M.W Kaluta, album cover art, poster art, long exposure, cel-shaded, character design 
cute cat anthro character, a highly intricate and hyperdetailed matte painting by Ismail Inceoglu, Junji Ito, James Gurney, Victo Ngai, Gregorio Catarino, M.W Kaluta and Kandinsky, album cover art, poster art 
a cute man wearing a fox mask, hypermaximalist mixed media ink illustration by Ismail Inceoglu, Sakiyama, James Gurney, Victo Ngai, Huang Guangjian and Zdzisław Beksiński, poster art, long exposure, cel-shaded, a masterpiece, octane render, digital artwork, ZBrushCentral, CGSociety 
a cute mouse with a war band costume, elemental art, uplifting, delightful, rutkowski, mucha and wlop glorious, sophisticated, gorgeous, distinctive lighting, charming, fantastical, particles, airy, formidable, complex, primary hues, splash screen 
portrait of a cute demon, by Ismail Inceoglu, Sakiyama, Lee Madgwick, Studio Ghibli, Victo Ngai, Michael Kaluta and David Friedrich, a masterpiece, ZBrushCentral, CGSociety, deep colours, photorealistic, sharp definition, HDR, octane render, 8k resolution, digital artwork, polychromatic, hypermaximalist, colorful, pixiv 
Handsome Powerful Hyper-Detailed! Portrait of Anthropomorphized!!! White Rabbit! Warrior in Intricate Formal Black Attire; Blue, Gold color scheme; by Greg Rutkowski, Matteo Scalero, Eytan Zana, Gustave Dore', Dan Mumford; Noble; Deep Colors; Hyper-Realism; 16K 
Full body character concept art of a girl next door in chibi style | | pixar - cute - fine - face, pretty face, realistic shaded perfect face, fine details by stanley artgerm lau, wlop, rossdraws, james jean, jakob eirich, andrei riabovitchev, marc simonetti, and sakimichan, trending on artstation 
female furry mini, cute, chibi, hedgehog girl, prickly spines, black red and white clothes, heart shape anime hair; character design conteset winner, character design contest winner, character adoptable, character design by Cory Loftis, Fenghua Zhong, Ryohei Hase, Ismail Inceoglu, Ruan Jia; 3d demo reel avatar, anime girl; style by maple story, zootopia; soft lighting, SFX, CGSociety 
Grogu, portrait, fantasy, medieval, vivid colors, elegant, concept art, sharp focus, digital art, hyper-realistic, 4k, unreal engine, highly detailed, hd, dramatic lighting by brom, trending on artstation 
a cute mouse with a war band costume, elemental art, uplifting, delightful, rutkowski, mucha and wlop glorious, sophisticated, gorgeous, distinctive lighting, charming, fantastical, particles, airy, formidable, complex, primary hues, splash screen 
a cute anthro wolf wearing a red hat :: by Ismail Inceoglu :: James Gurney :: Victo Ngai :: Cyril Rolando :: Michael Kaluta :: Makoto Shinkai :: elemental art :: uplifting :: delightful and wlop glorious :: sophisticated :: gorgeous :: distinctive lighting :: charming :: fantastical :: particles :: airy :: formidable :: complex :: primary hues :: splash screen :: a masterpiece :: 8k resolution :: Disney :: Pixar :: reimagined by industrial light :: character design 
the portrait of Mictlantecuhtli :: highly hypermaximalist matte photography :: Ismail Inceoglu :: James Gurney :: Victo Ngai :: Sakiyama :: Michael Kaluta :: Makoto Shinkai :: Gustave Doré :: elemental art :: matte :: uplifting :: delightful and wlop glorious :: sophisticated :: gorgeous :: distinctive lighting :: charming :: ∆ :: fantastical :: particles :: airy :: formidable :: complex :: primary hues :: splash screen :: a masterpiece :: 8k resolution :: radial gradient background :: reimagined by industrial light :: bright :: ZBrushCentral :: CGSociety 
highly detailed photograph of a cute man wearing a pumpkin mask :: pop art :: intricate and hyperdetailed painting :: Ismail Inceoglu :: Huang Guangjian :: Victo Ngai :: Roly Edwards :: Hergé :: Ed Wilmore :: Chris Blain :: Geoff McDowall :: Gediminas Pranckevicius :: CGSociety :: uplifting :: ZBrushCentral :: sophisticated :: gorgeous :: ∆ :: sharp crisp :: polished :: formidable :: primary hues :: complex :: cel-shaded :: album cover art :: a masterpiece :: 8k resolution : character design 
"Gothic Moon Witch" Beautiful; Tragic; Delicate; Detailed; Complex; Deep Colours; by Michael Kaluta; by Todd McFarlane ,by Tom Bagshaw; by Ralph Steadman; by Beeple; 
"Chibi Snail Girl" by Aeron Alfrey, Bernie Wrightson, Todd McFarlane, Boris Groh, Ralph Steadman, Dan Mumford and Michael Kaluta. Beautiful Sinister Horror, Cartoonish Realistic, Deep Colours, Hyperdetailed, 8k Resolution 
"Chibi FKA Twigs Voodoo Queen." by Aeron Alfrey, Bernie Wrightson, Todd McFarlane, Boris Groh, Ralph Steadman, Dan Mumford and Michael Kaluta. Beautiful Sinister Horror, Cartoonish Realistic, Deep Colours, Hyperdetailed, 8k Resolution. 
"Chibi Marilyn Manson as a Mad Scientist in Laboratory" by Aeron Alfrey, Bernie Wrightson, Todd McFarlane, Boris Groh, Ralph Steadman, Dan Mumford and Michael Kaluta. Beautiful Sinister Horror, Science-Fiction; Cartoonish Realistic, Deep Colours, Hyperdetailed, 8k Resolution. 
"Chibi Sia Furler" by Aeron Alfrey, Bernie Wrightson, Todd McFarlane, Boris Groh, Ralph Steadman, Dan Mumford and Michael Kaluta. Beautiful Sinister Horror, Alien, Science-Fiction, Cartoonish Realistic, Deep Colours, Hyperdetailed, 8k Resolution 
"Sinful Decadence - Pink Panther" Beautiful, Decadent, Sinister, Hyperdetailed, Eldritch; Deep Colours; Tom Bagshaw, Shintaro Kago, Roberto Ferri, Sandra Chevrier, Todd McFarlane, RHADS, Erik Johansson. Cinematic, 8K, Photorealistic. Octane Render. SFX. Ambiant Occlusion. Ray Traced 
"Portrait of a cute little flower spirit with big eyes", by Ismail Inceoglu, M.W. Kaluta, Anton Fadeev and Yoshitaka Amano, insanely detailed, 8k resolution, digital illustration, photorealism, trending on artstation, Vibrant Colours, delicate 
Edison Rex getting ready for Christmas, Artgerm, Pixar, Norman Rockwell, wlop, Coco, Luca, WLOP, intricately detailed concept art, 3d digital art, Maya 3D, ZBrush Central 3D shading, bright colored background, cinematic, Reimagined by industrial light and magic, 4k resolution post processing 
Renegade anthropomorphic rodent wearing a leather jacket riding a motorcycle, bespoke masterpiece, 35mm photoreal insanely detailed intricate, coherent blending, hyper detailed, extremely complex, trending on art station, octane render, hypermaximalist, deep color, neosurrealism by Thomas Kinkade Moebius Alphonse Mucha 
A chibi demon dressed up as an angel tric-or-treating:: by WLOP, by Fiona Staples, by Nicola Scott:: naively comics, 8k resolution , studio Lighting, colourful, Behance hd, ultradetailed 
Request:: by Peter Mohrbacher, by brian kesinger, by Mark Ryden:: Ultradetailed digital illustration, 32K resolution:: Deviantart, Behance HD:: steampunk illustration 
Two bros chilling inside one long log:: by Peter Mohrbacher, by brian kesinger, by Mark Ryden:: Ultradetailed digital illustration, 32K resolution, Pen and ink drawing, washed out colours:: Deviantart, Behance HD:: Steampunk illustration 
Photorealistic cheeky badger gambler dressed in a three-piece suit in a casino, antropomorphic, dim lighting, hyperrealistic, Ruan Jia, Jean Baptiste Monge, Ismail Inceoglu, artstation 
A seagull wearing a monocle and top hat:: by Peter Mohrbacher, by brian kesinger, by Mark Ryden:: Ultradetailed digital illustration, 32K resolution:: Deviantart, Behance HD:: steampunk illustration 
Purple worm smoking, overwhelmed by the shadows:: by Alexander Jansson, and Margaret Keane 
"portrait of a sad haunted soul of an unwanted boy", by Ismail Inceoglu, Yoshitaka Amano, Takehiko Inoue, Junji Ito, Dao Trong Le, insanely detailed, 8k, digital illustration, trending on artstation, delicate, sinister, complex, deep colours, Cinematic, Dramatic, Nightlight, Sharp focus, close-up 
Beautiful intricate wings made of blown-glass; an intricate realistic mixed media ink oil painting by Tom Bagshaw, Michael Kaluta, Beeple, Todd McFarlane, Ralph Steadman; photorealism, precisionism, ambient occlusion, a masterpiece, 3d liquid detailing, subsurface scattering, octane render, VRay, 16k 
northern forest troll, pointy ears, slouchy hat, shabby clothes; kim keever, swoon, edouard bisson, richard avedon, mark silvestri, johan egerkranz; earth tones, cinema 4k, 16k, octane, photorealistic, ultra-detailed, epic lighting, forest background, unreal engine 5, SFX, CGSociety, Artstation 
Extreme wide shot, 35mm, atmospheric urban photographic landscape, "a world with no future", giant smoke column billowing from a skyscraper on a wheat field, ground littered with pipes; concept art, gloomy, fantasy, scifi, fog, neon, cinematic; Michael Whelan, Vincent di Fate, Rutkowski; synthwave, vivid colors, dramatic, industrial 
"Sinister black rabbit stealing a metal tin of milk"; Scandinavian folklore, folklore creature, dramatic, fantasy art, illustration, ultrafine detailed painting, storybook illustration, colorized, elegant, sharp focus; Johan Egerkrans, Rembrandt, Jean Baptist Monge, Carlos Quevedo, Edouard Bisson, Jakub Różalski, Mark Silvestri, Marco Mazzoni; Album cover art, splatter paint, 3d liquid art, 16k, photorealistic, octane 
"Group of children sitting in a circle with an elaborate dreidle spinning in the middle of them"; Jewish history, Jewish stories, dramatic, fantasy art, illustration, ultrafine detailed painting, storybook illustration, colorized, elegant, sharp focus; Johan Egerkrans, Rembrandt, Jean Baptist Monge, Carlos Quevedo, Edouard Bisson, Jakub Różalski, Mark Silvestri, Marco Mazzoni; Album cover art, splatter paint, 3d liquid art, subsurface scattering, 16k, photorealistic, octane, unreal engine 
Cottagecore princess, gypsy skirt & puffy-sleeved blouse, gypsy wagon in background; dark fantasy, gloomy, moody, ultradetailed, storybook illustration, cinematic, bright, vivid colors, sketched; Alexander Jansson, Johan Egerkrans, Swoon, Edouard Bisson, Raffaello Ossola, Rovina Cai, Tetsuya Ishida; masterpiece, digital illustration, 16k, photorealistic, unreal engine, VFX, SFX, cgsociety, octane 
Little Red riding hood carrying a basket of bread and sweets through a forest with wildflowers on the ground; centered, ultradetailed, trending on wikiart, drawn by Andy Kehoe 
Humanoid reindeer family packing a sleigh with wrapped Xmas gifts; happy, bright, merry, vivid colors, storybook illustration, ink illustration; Alexander Jansson, Johan Egerkrans, Rovina Cai, Laurie Lipton, Tetsuya Ishida; album cover art, subsurface scattering, fairytale, character art, 8k 
An epic fairytale scene illustrated by Alexander Jansson and Johan Egerkranz 
A princess and a warty frog sitting at a koi pond in front of a fairy tale castle; art by Alexander Jansson 
A striking half portrait of an eldritch glowing god king, ichor, black antlers, fiery horned crown; moebius, hr giger, beksinski, artstation; digital art, 16k, hyperdetailed, photorealism, sharp focus, sfx, CGSociety, volumetric lighting, character concept art, clear face 
Inside the Goddess Head; Surreal, Beautiful, Tragic, Regal, Delicate, Nightmarish, Eldritch, hyperDetailed, Complex, Deep Colours; Tom Bagshaw, Michael Kaluta, Beeple, Todd McFarlane, Ralph Steadman; Horror Extremely Detailed, Symmetrical, Dark Fantasy, 16K, Character Concept, Industrial, Dark surrealism, Exquisite, octane 
Veiled Eldritch Being, Padma Mudra Pose, human face, tentacled; Beautiful; Tragic; Delicate, Nightmarish, Regal, Detailed, Complex, Deep Colours, vivid, bright, soft lighting, cinematic lighting, candlelit; Tom Bagshaw, Michael Kaluta, Beeple, Todd McFarlane, Ralph Steadman; 16k, octane, photorealism, cinematic 
intense portrait of kawaii chibi dryad girl, galaxy hair, ornate opal dress, sinister, creepy; vibrant, hyperrealism, hyperdetailed; in style of yoji shinkawa & hyung-tae kim; centered, artstation, dark fantasy, great composition, concept art, highly detailed, cinematic lighting, sharp focus, photorealism 
portrait of kawaii chibi dryad girl, short purple hair, ornate opal dress, sinister, creepy; vibrant, hyperrealism, hyperdetailed; in style of yoji shinkawa & hyung-tae kim; centered, artstation, dark fantasy, great composition, concept art, highly detailed, cinematic lighting, sharp focus, photorealism 
hand drawn cute gnome in disguise holding a bouquet of flowers, intricately detailed, hyperdetailed closeup of face, concept art, warm cinematic lightinig, volumetric, godrays, vivid, beautiful, colorful, artstation, Jordan Grimmer, huge garden scene, Greg Rutkowski, Philippe Druillet, Tomasz Alen Kopera, jean Baptiste Monge 
God Ganesh; creepy, grim, sinister, evil, melodramatic, ultradetailed, intricate, elegant, refined; Benedetto Fellin, Dan Witz, Rutkowski, Wadim Kashin, Junji Ito, artstation; 16k, photorealistic, celshaded, masterpiece, album cover art, digital illustration, poster art, vfx, sfx, unreal engine, raytraced 
Humanoid octopus in a dress; dark fantasy, gritty, gloomy, ultradetailed, illustration, colors of the sea; Trent Manning, Ghoulish Bunny Studios, Gris Grimley, Klopp Studio, Johan Egerkranz, Alexander Jansson, Swoon, Edouard Bisson; ethereal surrealism, surreal fantasy, 16k, photorealism, unreal engine, octane 
The beautiful cute Mosspunk Queen in her Serene Enchanted mossy covered forest. 8K UHR hyper-realistic digital watercolor, dark shadow, vray raycast light, realistic reflections & cell shading, fantasy, Unsplash, Pixar, beautiful vibrant light, Animation DO, Doga Kobo, Antonio Guidotti 
Face of a Beautiful cute glass-girl hybrid, ornate intricate gold crown erupting from head, glowing enchanted star, vivid shining gems abound. Extreme detail digital 3D HDR 8k vray fantasy Pixar beautiful vibrant light Cute Kawaii, Ufotable Inc. 
Face of beautiful Enchantress, inside The Great Library, behind her an Orangutan swings from a bookshelf with banana in hand. photorealistic digital hyper detail 3D HDR UHR 8k vray fantasy. Square Pictures, Sony Pictures, Terry Pratchett's Discworld 
Insert Subject Here, 8K 3D 8K resolution HDR, Dark shadows Vray Realistic Cell Shading Beautiful Raycast light, Ilya Kuvshinov, Cinema4d concept art fantasy 
the portrait of a magic alchemist with a fox mask, an intricate and hyperdetailed painting by Ismail Inceoglu and Dan Witz, fantasy art, deep colors, midjourney 
the magic golden tree filled with spells and incantations, a highly intricate and hyperdetailed matte painting by Ismail Inceoglu, Junji Ito, James Gurney, Dan Witz, M.W Kaluta and Kandinsky, fantasy art, album cover art 
the dark mark from the death eaters, a highly intricate and hyperdetailed matte photography by Alfonso Cuarón, album cover art, poster art, long exposure, cel-shaded, a masterpiece, a tattoo 
Surreal fantasy metal forest,neon, beautiful, hypermaximalist, unique bespoke, a masterpiece, impossible surreal architecture, warm fuzzy vibes, vibes everywhere, Ray Tracing Reflections, insanely detailed intricate, surreal, detailed digital matte, coherent blending, smooth geometric structure, vivid color, volumetric lighting, ambient occlusion,smooth UHD 
Impossibly beautiful cosmic goddess, stardust and particles, nebula complexion, intricate elegance, close up, Photoshop, cosmic goddess by Pär Olofsson, by Gustave Doré, Karol Bak, 35mm lens High Contrast, Kodachrome, Contre-Jour, Lumen Global Illumination, Ray Tracing Reflections, insanely detailed intricate, Unreal Engine 5 
Impossibly beautiful patchwork woman, made of a collage, galaxies moon glow stardust, gorgeous, pop art by Pär Olofsson, by Gustave Doré, Karol Bak, 35mm lens, Desaturated, High Contrast, Kodachrome, Contre-Jour, Lumen Global Illumination, Ray Tracing Reflections, insanely detailed intricate, Unreal Engine 5 
Impossibly beautiful cosmic goddess, stardust and particles, nebula complexion, intricate elegance, close up, Photoshop, cosmic goddess by Pär Olofsson, by Gustave Doré, Karol Bak, 35mm lens High Contrast, Kodachrome, Contre-Jour, Lumen Global Illumination, Ray Tracing Reflections, insanely detailed intricate, Unreal Engine 5 
Impossibly beautiful original fantasy painting, psychedelic mushroom kingdom, closed eyed visuals, epic scale, masterwork, masterpiece matte painting, insanely detailed intricate, coherent blending, hyper detailed, extremely complex, hypermaximalist, volumetric lighting, deep color, neosurrealism by Thomas Kinkade Moebius Alphonse Mucha 
Goddess close-up portrait ram skull, thorax, x-ray, backbone, jellyfish phoenix head, nautilus, orchid, skull, betta fish, bioluminiscent creatures, intricate artwork by tooth wu and wlop and beeple. octane render, trending on artstation, greg rutkowski very coherent symmetrical artwork. cinematic, hyper realism, high detail, octane render, 8k 
Avalon skull, close - up portrait, powerfull, intricate, elegant, volumetric lighting, scenery, digital painting, highly detailed, artstation, sharp focus, illustration, concept art, ruan jia, steve mccurry 
Full body portrait of a cute dryad girl with short blond and green hair wearing ornate green dress, vibrant colours. in style of yoji shinkawa and hyung-tae kim, trending on artstation, dark fantasy, great composition, concept art, highly detailed. 
A centered render of an mystical wild goddess with wearing ornate silver and gemstones and crystal clothing surrounded by flowing liquid gallium jellyfish and sacred geometry, perfect body and face, gorgeous, cinematic, beautifully lit, by alberto seveso, by zack snyder, by cory loftis, 3 d, trending on artstation, octane render, 8 k 
A photorealistic dramatic fantasy render of a beautiful woman wearing a beautiful intricately detailed japanese cat kitsune mask and clasical japanese kimono by wlop, artgerm, greg rutkowski, alphonse mucha, beautiful dynamic dramatic dark moody lighting, shadows, cinematic atmosphere, artstation, concept design art, octane render, 8k 
A full shot of a cute magical monster girl sea creature made of opals and tentacles. fully clothed. f1.4. symmetrical. dark smoke and vfx. caustics refraction. prism light. demon horns, angel wings, by giger and ruan jia and artgerm and range murata and wlop and william-adolphe bouguereau. pipes. lisa frank inspired. key art. fantasy illustration. award winning, artstation, intricate details, realistic, hyperdetailed, 8k resolution. 
the celestial clock who controls the universe, elemental art, uplifting, delightful, Victo Ngai, Makoto Shinkai, Studio Ghibli and wlop glorious, sophisticated, gorgeous, distinctive lighting, charming, fantastical, particles, airy, formidable, complex, primary hues, splash screen, anime key visual, a masterpiece 
the edge of the world :: highly hypermaximalist matte photography :: Ismail Inceoglu :: James Gurney :: Pablo Amaringo :: Sakiyama :: Michael Kaluta :: Makoto Shinkai :: Gustave Doré :: elemental art :: matte :: uplifting :: delightful and wlop glorious :: sophisticated :: gorgeous :: distinctive lighting :: charming :: ∆ :: fantastical :: particles :: airy :: formidable :: complex :: primary hues :: splash screen :: a masterpiece :: 8k resolution :: radial gradient background :: reimagined by industrial light :: bright :: ZBrushCentral :: CGSociety 
highly detailed portrait of a fire dragon :: pop art :: intricate and hyperdetailed painting :: Ismail Inceoglu :: Huang Guangjian :: Victo Ngai :: Roly Edwards :: Hergé :: Ed Wilmore :: Chris Blain :: Geoff McDowall :: Gediminas Pranckevicius :: CGSociety :: uplifting :: ZBrushCentral :: sophisticated :: gorgeous :: ∆ :: sharp crisp :: polished :: formidable :: primary hues :: complex :: cel-shaded :: album cover art :: a masterpiece :: 8k resolution : digital artwork :: character design :: fluid lines 
"Tentacles Wrapped Around Porcelain Doll" Beautiful; Delicate; Detailed; Complex; Fantasy; Nightmarish; Ethereal; Deep Colours; by Michael Kaluta; by Todd McFarlane ,by Tom Bagshaw; by Ralph Steadman; by Beeple 
"Eldritch Porcelain Doll" Beautiful; Delicate; Detailed; Complex; Fantasy; Nightmarish; Ethereal; Deep Colours; by Michael Kaluta; by Junji Ito; by Todd McFarlane ,by Tom Bagshaw; by Ralph Steadman; by Beeple 
"The Hermit" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman 
"Sinister Ornate Wheel" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman 
"The Fool" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman 
"The High Priestess" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman 
"Strength, Power" Tragic; Regal, Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman 
"The Magician" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman 
"Eldritch Being Wearing a Mask - Aztec Nightmare" Beautiful; Tragic; Delicate; Nightmarish; Regal; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman" 
"Veiled Eldritch Being - Padma Mudra Pose" Beautiful; Tragic; Delicate; Nightmarish; Regal; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman" 
"Masked Eldritch Creature - Padma Mudra Pose - Tendrils Tentacles" Beautiful; Tragic; Delicate; Nightmarish; Regal; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman" 
"Rotting Decaying Skeletal Bee Queen - Insect - Bees - Honey" Sinister, Decadent, Hyperdetailed Delicate; Complex; Eldritch; Deep Colours; Regal; Tom Bagshaw, Sandra Chevrier, Roberto Ferri, Michael Kaluta, Victor Adame Minguez, Ruan Jia, Alphonse Mucha, Erik Johansson. Cinematic, 8K, Photorealistic. Octane Render. SFX. Ambiant Occlusion. Ray Traced 
"Eldritch Decadence - Brain Casing, Cerebral Corona, Sinister Cortex" Beautiful, Heavenly; Delicate; Defined; Sinister, Hyperdetailed, Deep Colours; Shintaro Kago, Tom Bagshaw, Sandra Chevrier, RHADS, Junji Ito, Erik Johansson. Cinematic, 8K, Photorealistic. Octane Render. SFX. Ambiant Occlusion. Ray Traced 
Beautiful intricate gown made of blown-glass worn by a fairy; an intricate realistic mixed media ink oil painting by Ismail Inceoglu, Junji Ito, Ian Miller, Bosch, Dali, Pino Daeni; photorealism, precisionism, ambient occlusion, a masterpiece, 3d liquid detailing, subsurface scattering, octane render, VRay, 16k 
"Fragmented Supernova - Fractured Shards of Crumbling Beauty" Beautiful; Voxel Fractals; Decadent; Hyper Detailed; Cubic; Complex; Fragments; Surreal; Eldritch; Deep Colours; Cinematic; 8K; Photorealistic; By Tom Bagshaw; Ryohei Hase; Junji Ito; Sandra Chevrier; Beeple; Ralph Horsley; Roberto Ferri; Michael Kaluta; Victor Adame Minguez; Todd McFarlane; Alphonse Mucha; Erik Johansson. Octane Render; SFX; Ambient Occlusion; Photorealistic 
Beautiful dark flooded landscape, giant human skull in the color of the universe, in the style of beeple and mike winkelmann, photo real, ultra realistic, intricate, epic lighting, 8k resolution, unreal engine 5, ultraviolet colors 
Double exposure of dally life, symbols of live, explosion, cyber mushroom city, love is the most relevant theme, love is infinity, love is begin of all, 8 k resolution, artistic mode, artistic, trending on instagram, long exposure, love art, serious, fantasy and dreams vibes, mushrooms style and macro style 
Hyper detailed 3d render like a oil painting - the secret garden by jacek yerka, mariusz lewandowski, houdini algorithmic generative render, abstract brush strokes, masterpiece, edward hopper and james gilleard, zdzislaw beksinski, mark ryden, wolfgang lettl, hints of yayoi kasuma, octane render, 8k 
Mysterious ancient dieties hovering over magical glass temple, infinite quantum waves, synthwave, bright neon colors, highly detailed, cinematic, tim white, michael whelan, roger dean, bob eggleton, lisa frank, vladimir kush, kubrick, kimura, isono 
Dark fantasy oil painting, giant bonsai, big river, bloodmoon, floating mountains, volumetric light, misty, godrays, high details, 8 k, beksinski, strange creatures, eerie 
Portrait of a cute and adorable fox wizard, wearing tailcoat, steampunk, anthromorphic, by Jean Baptiste monge, Karol Bak, victorian character, album art, character design, detailed painting, hyperdetailed, hyperrealism, a masterpiece, by Greg Rutkowski, trending on artstation 
Portrait of a cute and adorable fox wizard, wearing tailcoat, steampunk, anthromorphic, by Jean paptiste monge, Karol Bak, victorian character, album art, character design, detailed painting, hyperdetailed, hyperrealism, trending on artstation 
Tim burton Alice girl falling into a steampunk world of Oz by Ismail Inceoglu, Cyril Rolando, Sir John Tenniel, Josephine Wall, Dino Tomic, 3d render by Alfons Mucha , hyperdetailed Behance HD CGSociety 8K resolution 
"Portrait of a cute little lion astronaut in love", by Ismail Inceoglu, M.W. Kaluta, Anton Fadeev and Yoshitaka Amano, insanely detailed, 8k resolution, digital illustration, trending on artstation, delicate, close-up 
"Soulless Devourer”, Intricate, elaborate transparent glass sculpture on fire, Scenic, Hyperdetailed, Dora Maar, Bagshaw, Chevrier, Ferri, Kaluta, Amy Sheila, Jane Jones. Cinematic, WLOP, 8K, concept, sharp focus, rutkowski, Artgerm, Octane render, a masterpiece, volumetric lighting 
forest fair, cuteness, kawai, a photography by WLOP, Dragan Ilic Di Vogo, Bartosz Domiczek, Ivan Shishkin, erin hanson, ismail inceoglu, deep colors, fantasy art, sharp focus, magical, fable, stunning, beautiful, a masterpiece, album cover art, deviantart, vray 
"Humanoid Feline Cat in Regal Robes Praying Under the Moon" Gorgeous Mixed Media Gouache Full-length Portrait by Ruan Jia, Jimmy Law, Peter Mohrbacher, Pino Daeni, Victo Ngai, Sandra Chevrier, Moebius and Peter Max; Intricate; Elegant; Watercolour; Ethereal 
"Humanoid Owl Wearing Regal Robe Standing in Front of Full Moon" Gorgeous Mixed Media Gouache Full-length Portrait by Ruan Jia, Jimmy Law, Peter Mohrbacher, Pino Daeni, Victo Ngai, Sandra Chevrier, Moebius and Peter Max; Intricate; Elegant; Watercolour; Ethereal 
The Phoenix:: by WLOP, by Jon Foster, by Michael Vincent Manalo:: Comic Book, Ultra-HD, studio Lighting, colourful, Photorealism, ultradetailed:: Energy, Joy, Rebirth, fire, power, taking flight 
Kawaii Humanoid Cat, Arcane Glow, Druidic Runes, Dramatic, Intricate, Human Cat Face, Cat Ears, Smooth, Sharp Focus, Illustration, Octane, Leesha Hannigan, Ross Tran, Thierry Doizon, Kai Carpenter, Ignacio Fernando Rios, artstation, concept art, fantasy art, hyperdetailed, hyperrealism 
Beautiful intricate gown made of blown-glass worn by a fairy; an intricate realistic mixed media ink oil painting by Ismail Inceoglu, Junji Ito, Ian Miller, Bosch, Dali, Pino Daeni; photorealism, precisionism, ambient occlusion, a masterpiece, 3d liquid detailing, subsurface scattering, octane render, VRay, 16k 
"Illusions and Reality”, Intricate, light yellow and orange, Scenic, Hyperdetailed, Royo, Unusual Colors, Bagshaw, Sandra Chevrier, Michael Ferri, Michael Kaluta, Cinematic, WLOP, 8K, concept, smooth sharp focus, greg rutkowski, Artgerm, Giger, (liminal space) 
surreal dreamscape, Spectre, wraith-like demon sitting on stomach of dreamer, night terrors, dark, trinkets & adornments of skulls and bones; Johan Egerkranz, Edouard Bisson, Calas Lonut; soft lighting, 16k, photorealistic, ultra-detailed, cinema 4D, unreal engine 5, SFX, CGSociety, Artstation 
"Death Angel", thorny crown, tears of tar"; beautiful, sinister, regal, eldritch; hyperdetailed, deep colors, vivid; Tom Bagshaw, Shintaro Kago, Ferri, Chevrier, Todd McFarlane, Erik Johansson; cinematic, 16k, octane, SFX, album cover art, photorealistic, ambient occlusion, ray-traced, unreal engine 5 
"Stop breathing", intricate, sinister, elegant, digital, scenic, hyperdetailed, complex; Luis Royo, Eldritch, Tom Bagshaw, Chevrier, Ferri, Kaluta, Victor Adame Minguez, Ruan Jia; deep colors, vivid, cinematic, 16k, photorealistic, blender 3d, vaporwave, surrealism 
Evil Tomte; dark fantasy, beautiful, colorful, intricately detailed, ink illustration; Eldritch, Johan Everkranz, Jean Baptiste Monge, Carlos Quevedo, Edouard Bisson, Marc Silvestri, Marco Mazzoni; album cover art, 3d liquid art, subsurface scattering, octane, 16k, photorealism 
anaglyph of an evil lich king; Hayao Miyazaki, Bosch, Giger, Pieter Bruegel the Elder, Alex Grey; digital art, dark fantasy, terrifying, horror, nightmare, sophisticated, stunning, epic lighting, film grain, golden hour, 16k, photorealism, Unreal Engine, Environment Design 
"portrait of a scarlet-colored demon", 7 heads, 10 horns, cloaked woman on its back; photographic, masterpiece; Gustave Dore, Sam Spratt, Allen Williams, artstation; CGSociety, 16k, photorealism, earth-tone colors, unreal engine, SFX 
Grotesque humanoid cryptid bog creature; dark fantasy, symmetrical, plant armor, intricate, hyperdetailed, golden hour, woodland swamp, stylized, sketchbook, ink illustration, splatter paint; Brian Froud, Johan Egerkranz, Swoon; album cover art, masterpiece, poster art, 16k, photorealism, unreal engine, octane 
Award winning national geographic photograph of an ultra-disturbing creepy creature, creapy disturbing smile, sharp detailed teeth, grotesque, wet; photorealistic; Gustave Doré, Rutkowski; 32k, concept art, ultra-detailed, Eldritch, artstation, unreal engine, 64 megapixels, back lit, 3d shading, cel-shaded, gouache painting 
Alice in wonderland, eating a bloody heart; death tarot card, ul 
tradetailed, photorealistic, half skull face, cinematic, 16k; stanley artgermm, Johan Egerkranz, Tim Burton, tom bagshaw, greg rutkowski, carne griffiths, ayami kojima, beksinski, giger, trending on deviantart; intricate, horror, colorful, vivid, bright 
Alice in wonderland, holding a pet skull, tattered clothes; ultradetailed, photorealistic, skull face, cinematic, 16k; stanley artgermm, Johan Egerkranz, Tim Burton, tom bagshaw, greg rutkowski, carne griffiths, ayami kojima, beksinski, giger, trending on deviantart; intricate, horror, colorful, vivid, bright, bloody 
Realistic human skull, rotting; sinister pop surrealism, lowbrow art, sultry grotesque, soft lighting; rococo, swoon, edouard bisson, richard avedon, natalie shau, loreta lux, tom bagshaw, mark ryden, trevor brown; 16k, photorealism, SFX, CGSociety 
Rotting skeleton queen covered in spider swarm, crown; Sinister, pop surrealism, lowbrow art, grotesque, Hyperdetailed,, intricate, Eldritch, vivid; rococo, swoon, edouard bisson, richard avedon, natalie shau, loreta lux, tom bagshaw, mark ryden, trevor brown; 16k, photorealism, SFX, CGSociety 
"Unholy Humanoid Sphynx Cat", demonic, satanic robes; damnation, hell, fear, evil, horror, fire, gloomy, horror; ultradetailed; artgerm, Dwayne Barlow, artstation, Greg Rutkowski, Mucha; 16k, photorealism, unreal engine, SFX, CGSociety, vaporwave 
"(Portrait of a scarlet-colored demon)", 7 heads, ((10)) horns; photographic, masterpiece, album cover art, digital art, subsurface scattering, splatter paint; Gustave Dore, Sam Spratt, Allen Williams, artstation; CGSociety, 16k, photorealism, earth-tone colors, unreal engine, SFX 
"A beautiful & terrifying new creature from folklore", smoke & shadow, glowing eyes & mouth; clear detailed view, ethereal fantasy art, dark fantasy, cinematic, beautiful lighting; Greg Rutkowski, Carne Griffiths, Peter Mohrbacher, Tomasz Alen Kpera, Fenghua Zhong, Beksinski; 16k, photorealism, unreal engine, SFX, octane, celshaded 
"front view portrait of an impossibly beautiful, angry female demon", perfectly centered, rotting flesh, growing ornamentation; ornate, ultradetailed, symmetrical, elegant, beautifully soft lit; Wane Barlowe, Peter Mohrbacher, Kelly McKernan; 16k, photorealism, unreal engine, SFX, CGSociety, octane, raytracing, stunning scene, artstation 
"Cheshire cat death tarot card", skull face; ultradetailed, cinematic, 16k, photorealism; Artgerm, Tom Bagshaw, Greg Rutkowski, Carne Griffiths, Ayami Kojima, Beksinski, Giger, trendng on Deviantart; horror, colorful, vivid, bright, SFX, CGSociety, unreal engine, octane; Lisa Frank 
"Translucent wendigo with flaming eyes"; misty, heroic lighting, dark fantasy, intricate, ultradetailed, photorealistic, digital painting, artstation, illustration, concept art, smooth, sharp focus; John Collier, Albert Aublet, Krenz Cushart, Artem Demura, Alphonse Mucha; 16k, photorealism, unreal engine, SFX, CGSociety, celshaded, raytraced 
scary evil Swedish tomte; a beautifully colorful and intricately detailed mixed media ink illustration by Johan Egerkrans, Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Eldritch, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
queen of death & decay, grotesque, meaty, bloodbath, skulls; ultra-detailed, hyperrealistic, mixed media painting, technical pen; Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Calas Lonut; album cover art, photorealism, 16k, 4d liquid detailing, octane, vivid 
technical pen, centered, grotesque, zombiecore gourd-gnome, vintage; a beautifully colorful & intricately detailed mixed media ink illustration; Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, & Tetsuya Ishida; album cover art, 3d liquid art, subsurface scattering, octane render, 16k, vivid, hyperrealism 
technical pen, centered, grotesque, zombiecore tomte gnome, vintage; a beautifully colorful & intricately detailed mixed media ink illustration; Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, & Tetsuya Ishida; album cover art, 3d liquid art, subsurface scattering, octane render, 16k, vivid, hyperrealism 
a grotesque zombiecore face; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Haunted doll with glowing eyes and tattered clothes, by Asaf Hanuka, Anna Dittmann, ink drawing, fantasy art, concept art, storybook illustration, hyperrealism, hyperdetailed, 16k, octane render, vivid, bright, colorful 
Scene magic silent hill eldritch horror; a beautifully colorful dark and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Scene magic nightscape eldritch horror; a beautifully colorful dark and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Scene magic dark nightscape eldritch horror; a beautifully colorful dark and intricately detailed ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, masterpiece, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Creepy horror poster illustration, little imp with a tail and cloven feet playing a wooden six holed flute, dark horror, surreal, illustration, black ink on old parchment paper, vintage, Ally Burke, Steven Stifler, Thomas kinkade, Ralph steadman 
Scene water magic silent hill horror; a beautifully colorful dark and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Scene water magic silent hill creature; a beautifully colorful dark and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Scene water magic silent hill; a beautifully colorful dark and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Bugs Bunny, zdsislaw beksinski, grotesque, eldritch horror, extremely dramatic lighting, candle lit, 8k, tendrils growing around, black, darkness, black slime tendrils, many tendrils, infected, body horror, Bugs Bunny, Bugs Bunny face, horror, octane render 
line & wash detailed 3 headed hellhound; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Scene dark magic silent hill; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Formal Portrait of holiday Krampus; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
grotesque zombiecore white magic kawaii beautiful witch; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
scary evil Swedish tomte; a beautifully colorful and intricately detailed mixed media ink illustration by Johan Egerkrans, Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Eldritch, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
two grotesque skeletons kissing madly in love; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Eldritch, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
two grotesque skeletons embracing each other; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Eldritch, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
a grotesque anatomical cross-section of a monster; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
portrait of a grotesque ogre; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Eldritch, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
portrait of a Scandinavian troll; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Eldritch, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
creepy haunted house covered in mist inhabited by spirits; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
a grotesque scary creepy beast of epic proportions; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
a grotesque decaying heart in a glass bulb; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
grotesque zombiecore hand and skeleton hand draw eachother; a beautifully colorful and intricately detailed mixed media ink illustration by Escher, Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
a grotesque zombiecore hand with eyes walking on fingers; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
a grotesque zombiecore head exploding; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Specifications diagram of a honey monster cross section; a beautifully intricate and detailed mixed media ink illustration by Ismail Inceoglu, Junji Ito, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, album cover art, a masterpiece, 3d liquid detailing, subsurface scattering, octane render, VRay, 16k resolution 
Honey monster anatomical cross-section; a beautifully intricate and detailed mixed media ink illustration by Ismail Inceoglu, Junji Ito, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, album cover art, a masterpiece, 3d liquid detailing, subsurface scattering, octane render, VRay, 16k resolution 
Monster anatomical cross-section; a beautifully colorful and detailed mixed media ink illustration by Ismail Inceoglu, Junji Ito, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, album cover art, a masterpiece, 3d liquid detailing, subsurface scattering, octane render, VRay, 16k resolution 
Fish skeleton; A Beautifully Colorful And Detailed Mixed Media Ink Illustration By Ismail Inceoglu, Junji Ito, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, Album Cover Art, 3d Liquid Detailing, Subsurface Scattering, Octane Render, VRay, 16k Resolution, Vivid, Asaf Hanuka, Anna Dittmann, Hyperdetailed, hyperrealism 
Summoner, by Greg Rutkowski, Giger, Gustave Doré, Wadim Kashin, hyperdetailed Eldritch, 8k, concept art, 3D shading, 3Delight, 3ds Max, CGSociety, CryEngine, Horde3D, Octane Render, Substance Designer, shadow depth, VRay, ZBrush, hyperrealism, trending on Artstation 
anaglyph of an evil lich king; Hayao Miyazaki, Bosch, Giger, Pieter Bruegel the Elder, Alex Grey; digital art, dark fantasy, terrifying, horror, nightmare, sophisticated, stunning, epic lighting, film grain, golden hour, 16k, photorealism, Unreal Engine, Environment Design 
"Zombiecore rotting decaying eagle", sinister, nightmarish; ultra-detailed, elegant, regal, complex, eldritch, bold colors; Tom Bagshaw, Sandra Chevrier, Ferri, Kaluta, Victor Adame Minguez, Ruan Jia, Mucha, Erik Johansson; cinematic, 16k, photorealistic, octane, unreal engine, SFX, ambient occlusion, ray traced 
"Gaping maw!!!! Filled with bloody teeth!! Teethcore"; sinister, nightmarish, close-up portrait, intricate, volumetric lighting, detailed gouache painting, sharp focus, photorealistic; Ruan Jia, Giger, Mucha; grotesque, 16k, deep colors, unreal engine, octane 
Disgusting disturbing dutch golden age bizarre mutant portrait made from flower floral still life with many human tongues, ultra-detailed fungus, bizarre, slimy, forms sprouting up everywhere; rachel ruysch, christian rex van minnen, dali; black background; chiaroscuro; dramatic lighting, perfect composition, oranate, 8k  
oil painting, black background; christian rex van minnen, dali, todd schorr; chiaroscuro portraiture; extremely bizarre, disturbing; mutated man with intense acne obscuring features; volumetric lighting, perfect composition, recursive, masterpiece, ornate, 16k, ultra-detailed, photorealism 
ceremonial mask; hyperrealistic, digital painting; Wane Barlowe, Beksinski, Giger, Austin Osman Spare, bussiere, Roberto Ferri, Rembrandt, Luis Racardo Falero; intricate ornamentation, 32k, photorealistic, unreal engine, octane 
"I have no mouth but I must scream"; digital painting, surreal, dark, grim, dark fantasy; Hieronymus Bosch, Rutkowski, Anna Podedworna; ultra-detailed, elegant, intricate, sharp, photorealistic, 32k, unreal engine, octane, ornate background scenery, well-composed 
Pale husk soulless woman, dehydrated skin, mantis eyes, agony expression, gorgeous, beautiful; grotesque fantasy character, full portrait, ultra-realistic, extremely intricate, highly detailed, crispy quality; Mohrbacher, Hajime Sorayama, Wane Barlowe, Boris Vallejo; 8k, 16k, photorealistic, unreal engine, octane 
Impossibly grotesque patchwork woman, made of collage, gorgeous, meat, skin, bones, bloodbath; Pär Olofsson, Gustave Doré, Karol Bak; 35mm lens, Full-Color, High Contrast, Kodachrome, Contre-Jour, Lumen Global Illumination, Ray Tracing Reflections, ultradetailed intricate, Unreal Engine 5 
Alice in wonderland, eating a human heart, death tarot card, ultradetailed, half skull face, cinematic, 16kk; artgerm, tom bagshaw, greg rutkowski, carne griffiths, ayami kojima, beksinski, giger, trending on deviantart;photorealistic, horror, full of colour, SFX, CGSociety, HD 
evil lich king; Hayao Miyazaki, Bosch, Giger, Pieter Bruegel the Elder, Alex Grey; digital art, dark fantasy, terrifying, horror, nightmare, sophisticated, stunning, epic lighting, film grain, golden hour, 16k, photorealism, Unreal Engine, Environment Design, anaglyph 
Panoramic award winning national geographic photograph of a sexy bloated old wrinked BBW deformed (pig-human hybrid) with (many eyes) and (humongous boobs) and (((a giant penis))), high details; obscene; lurid; grotesque; colorful; pornography photography by nicoletta ceccoli, dan witz, ren hang, bbernie wrightson, Disney's and artgerm;  
full-length photography of a hideous bloated old wrinkled deformed BBW (man-pig hybrid) with (pink tentacles) and (large breasts) and (((a giant penis))), high details; obscene; llurid; grotesque; colorful; pornography photography by Nicoletta Ceccoli, Dan Witz, Ren Hang, Bernie Wrightson, Disney's and Argerm 
"Portrait of Pinhead", Hellraiser, by Ismail Inceoglu, Yoshitaka Amano, Takehiko Inoue, Junji Ito, Dao Trong Le, insanely detailed, 8k, digital illustration, trending on artstation, delicate, sinister, complex, soft, deep colours, Cinematic, Dramatic, Nightlight, Lumen Reflections, Sharp focus, close-up 
"A glow in the dark!!! ghosts!!", by Ismail Inceoglu, Yoshitaka Amano, Takehiko Inoue, Junji Ito, Dao Trong Le, insanely detailed, 8k, digital illustration, trending on artstation, delicate, sinister, complex, soft, deep colours, Cinematic, Dramatic, horror, terrifying, Sharp focus 
Cosmic psychedelic masterpiece 35mm , landscape, beautiful, hypermaximalist, unique and bespoke, brilliant horror neosurrealism by Andreas Franke and Ted Nasmith, Cosmic matte by Leon Tukker and Mamoru Oshii, stunning photorealism by Ernst Haas and Max Dupain, insanely detailed intricate, coherent blending, vivid color, UHD 
Athanasius Kircher; demonology; M.C. Escher; journal; polymath; encyclopedia; William Blake; pseudepigraphical grimoire; Key of Solomon; illuminated manuscript; ratios; occult; codex; macabre; alchemy; apothecary; vials; potions; diagram; schematics; heliograph: Luigi Serafini; Harald Metzkes; Ernst Fuchs; Wlastimil Hofman; exquisite; masterpiece; figure drawing; cabaret; cult of Dionysus; alien botany book; structural formula; math equation; calculus; graph; skill tree; ecological biopunk 
Thomas the tank engine in style of zdzisław beksinski, extremely dramatic lighting, 8 k, tendrils, black, darkness, black slime tendrils, infected, rust, body horror, thomas the train, thomas the tank engine face, horror 
Detailed portrait zombie, cyberpunk futuristic neon, reflective puffy coat, decorated with traditional japanese ornaments by ismail inceoglu dragan bibin hans thoma greg rutkowski alexandros pyromallis nekro rene maritte illustrated, perfect face, fine details, realistic shaded, fine-face, pretty face 
Urban school zombie girl in tattered clothes fanart, dark blue long hair, muted colors, matte print, pastel colors, ornate, digital art, cute smile, digital painting, fan art, elegant, pixiv, by ilya kuvshinov, by studio ghibli 
Ghostly woman peering out from the curtains; ominous, surreal, gloomy, foggy atmosphere, full body image, hyperdetailed, hyperrealistic, dark, horror; Mark Arian, Zdzislaw Beksinski, Santiago Caruso; 16k, octane, SFX, CGSociety 
A strange entity creature lurks in the shadows of an alleyway, digital art by samuel araya, chronicles of darkness art, god machine chronicle art 
"Pale Demonic Woman in White Robes With Black Curved Horns. Ethereal Moonlight." Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman 
"Eldritch Being Wearing a Mask - Aztec Nightmare" Beautiful; Tragic; Delicate; Nightmarish; Regal; Detailed; Complex; Deep Colours; by Tom Bagshaw, Michael Kaluta, Beeple, Todd McFarlane, Ralph Steadman, Dan Witz and Erik Johansson 
"Undead Rotting Decaying Griffin" Sinister, Nightmarish; Hyperdetailed Delicate; Complex; Eldritch; Deep Colours; Tom Bagshaw, Sandra Chevrier, Roberto Ferri, Michael Kaluta, Victor Adame Minguez, Ruan Jia, Alphonse Mucha, Erik Johansson. Cinematic, 8K, Photorealistic. Octane Render. SFX. Ambiant Occlusion. Ray Traced 
"Smiling!! Creepy! Old Hag with Glowing! Eldritch Eyes"; Gorgeous Sinister Close-up Portrait by Antonio José Manzanedo, Seb McKinnon and Ralph McQuarrie; Intricate; Volumetric Lighting; Detailed Gouache Painting; Sharp Focus; Artstation; Grotesque Nightmare 
"Gelatinous!!! Woman! :: Slug! Features :: Slimy Jelly"; Gorgeous Sinister Close-up Portrait by Antonio José Manzanedo, Seb McKinnon and Ralph McQuarrie; Intricate; Volumetric Lighting; Detailed Gouache Painting; Sharp Focus; Artstation; Grotesque Nightmare 
"portrait of gelatinous woman!", slug features, slimy jelly; gorgeous, sinister, intricate, ultra-detailed; Antonio José Manzanedo, Sem McKinnon, Ralph McQuarrie; volumetric lighting, detailed gouache painting, sharp focus, grotesque, nightmarish, photorealistic  
"Rat Swarm Mother - Swarm of Rates - Hideous Fur - Grotesque Wicked - Rat Crown" Beautiful, Decadent, Sinister, Hyperdetailed, Eldritch; Deep Colours; Tom Bagshaw, Shintaro Kago, Roberto Ferri, Sandra Chevrier, Todd McFarlane, RHADS, Erik Johansson. Cinematic, 8K, Photorealistic. Octane Render. SFX. Ambiant Occlusion. Ray Traced 
"Diamonds Shards in Bleeding Eyeholes" Sinister; Disturbing; Gruesome; Hyper Detailed; Bloody; Complex; Misery; Suffering; Eldritch; Deep Colours; Cinematic; 8K; Photorealistic; By Tom Bagshaw; Ryohei Hase; Junji Ito; Sandra Chevrier; Beeple; Ralph Horsley; Roberto Ferri; Michael Kaluta; Victor Adame Minguez; Todd McFarlane; Alphonse Mucha; Erik Johansson. Octane Render; SFX; Ambient Occlusion; Photorealistic 
"Skinless Johnny Depp - Necrotic Rotten - Flesh Eating Virus" Photorealistic; Sinister; Disturbing; Gruesome; Hyper Detailed; Bloody; Complex; Suffering; Eldritch; Deep Colours; Cinematic; 8K; Photorealistic; by Jenny Saville, Nychos, Sandra Chevrier, Dan Witz, Ernst Haeckel, Alphonse Mucha, Chiharu Shiota, Max Ernst, Tom Bagshaw, RHADS and Erik Johansson. Octane Render; SFX; Ambient Occlusion 
"Creepy!!! Lunatic! Insane!! Doctor! in Blouse!", sharp fangs, toothy smile; Gorgeous Sinister Long Shot Portrait by Antonio José Manzanedo, Seb McKinnon and Ralph McQuarrie; Intricate; Volumetric Lighting; Detailed Gouache Painting; Sharp Focus; Artstation; Grotesque Nightmare 
"Gaping Maw!!!!! Filled With Bloody Teeth!!"; Gorgeous Sinister; Close-Up Portrait; Intricate Elegant; Volumetric Lighting; Detailed Digital Painting; Sharp Focus; Ruan Jia; HR Giger; Alphonse Mucha; Grotesque Nightmare; Photorealistic; Deep Colours; 
Bizarre anatomical zombiecore cross-section; a beautifully intricate and detailed mixed media encaustic painting, ink on newspaper, by Ismail Inceoglu, Junji Ito, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, album cover art, a masterpiece, 3d liquid detailing, subsurface scattering, octane render, VRay, 16k resolution 
A crooked and jagged matchstick, splintered wood, Burned and charred texture, Inked art, by Ralph Steadman, Hand-Drawn, Crosshatch, Ink, Black and White, 8K, insanely detailed and intricate, elegant, ornate, hyper-realistic, super detailed 
Strange mystical and esoteric creatures floating in a scary void environment, intricate, Highly Detailed, Webbing, ornate, skulls, dark, concept art, 4k, inspired by Reylia Slaby, Frank Frazetta, trending on artstation, 5D, Dust particles, volumetric lighting, CGsociety, Blender Render 
detailed digital art of a zombie champion king completely wrapped in bandages and fabrics; magic the gathering art by Volkan Baga, rk post, Raymond Swanland, artstation 
Dark Ambient lighting, Alien Nest, Creepy Sticky, leaking, h.r.giger. DREAM, Nightmare, dimly lit, centered composition., by Charlie Bowater, Marker Art, Concept Art, volumetric Light, Paper 
Broken Eggs, Dark Ambient lighting, Alien Nest, Creepy Sticky, leaking. DREAM, Nightmare, dimly lit, centered composition., by Charlie Bowater, Marker Art, Concept Art, volumetric Light, Paper, lurking creature, dusty, cave, ink drawing biopunk 
Epic witchcraft at the parallel world in complementary colors, depicted by Kael Ngu, Ivan Shishkin, Kim Jung Gi, Shepard Fairey and Samantha Cheevers 16k resolution, hyperrealism maximalistic, beautiful spectacular striking textures, amazing picturesque eye-catching coloration, impressive magnificent setting, remarkable dramatic clouds, unforgettable stunninggolden point, great composition 
Highly detailed haunted castle hidden below a dark tree and overgrown vines and flowers, in an overgrown sinister jungle with dark flowers vines and greenery, fine art by Aeflus Ben Durandt, Masterpiece Epic Ominous, Whimsical, deviantart 
Zombie juggalo furrie hybrid, oil on canvas James Gurney Wadim Kashin Kevin Sloan Martin Wittfooth Annie Stegg 
creepy vintage family portrait, by greg rutkowski, Benedetto Fellin, dan witz, peter mohrbacher, Wadim Kashin, junji ito, ultra detailed, hyperdetailed, intricate and refined, cel-shaded, trending on artstation, a masterpiece, post-production; vfx, album cover art, poster art 
dozens of people sucked up to the sky, by Jiajun Wang, Greg Rutkowski, Junji Ito, finnian macmanus, dark fantasy art, hyperdetailed, intricate and refined, cel-shaded, deviantart, a masterpiece, album cover art 
a cursed artefact, Splatter Paint, Modern Art, Concept Art, Double-Exposure, Cool Color Palette, Backlight, Ray Tracing Reflections, insanely detailed and intricate, hypermaximalist, elegant, ornate, hyper realistic, super detailed, sinister, deviantart 
"Swine Cthulhu Barbie Kitten Fetus Embryo Placenta" Dirty Grim Mixed Media Gouache Watercolour by Ruan Jia, Masahiro Ito, Boris Groh, Francis Bacon, Pino Daeni and Richard Dadd; Dirty; Brownish; Grotesque; Intricate; Surreal Horror; Coherent, Ethereal, Unfinished 
"Flayed Catwoman Skinless Skinned Decadent" Mixed Media Gouache Painting by Bastien Deharme, Ruan Jia, Pino Daeni and Ray Caesar; Deep Colours; Grotesque; Intricate; Abstract Horror; Coherent 
"Exploding Body!! Blood Splatter" Mixed Media Gouache Painting by Jenny Saville, Nicoletta Ceccoli, Victor Adame, Erik Johansson, Francis Bacon, Dave McKean, Dan Witz and Santiago Caruso; Deep Colours; Grotesque; Intricate; Abstract! Horror 
"Pygmy Totem Pole Sacrifice Innards Organs Viscera" Mixed Media Gouache Painting by Jenny Saville, Nicoletta Ceccoli, Victor Adame, Erik Johansson, Francis Bacon, Dave McKean, Dan Witz and Santiago Caruso; Deep Colours; Grotesque; Intricate; Abstract Horror 
"Ribs!! Spine!! Guts Bloody Innards Organs" Mixed Media Gouache Painting by Jenny Saville, Nicoletta Ceccoli, Victor Adame, Erik Johansson, Francis Bacon, Dave McKean, Dan Witz and Santiago Caruso; Deep Colours; Grotesque; Intricate; Abstract Horror 
"Faceless!! Creature Decadence! Macabre Delights"; Detailed Watercolour Painting by Jenny Saville; by Francis Bacon; by Mariusz Lewandowski; by Santiago Caruso; Rich Colours; Sharp Focus; Grotesque Nightmare; Intricate; Deep Colours 
"Skinned Skinny Skinless Body Meat Hooks! Butcher"; Detailed Matte Oil Painting by Jenny Saville; by Francis Bacon; by Mariusz Lewandowski; by Santiago Caruso; Rich Colours; Sharp Focus; Grotesque Nightmare; Intricate 
"Green Pale Spiralling Vortex!!! of Void!!"; Vortex by Antonio José Manzanedo, by Seb McKinnon, by Ralph McQuarrie; Detailed Gouache Painting; Focused; Deep Colours; Textured; Meticulous 
"Sinister!!! Mutated! Decaying! Eye!! Flayed!!! Feral Gelatinous! Eyeball!!"; Gorgeous Eye by Antonio José Manzanedo, by Seb McKinnon, by Ralph McQuarrie; Detailed Gouache Painting; Focused; Grotesque; Disturbing; Deep Colours; Textured; Meticulous 
"Huge Eyeball!! of Sheep! on Long! Neck! Slimy!! Tar!! Bubbling! Wet!"; Gorgeous Gruesome Portrait by Antonio José Manzanedo, by Seb McKinnon, by Ralph McQuarrie; Detailed Gouache Painting; Focused; Grotesque; Disturbing; Deep Colours; Textured; Meticulous 
"Huge Eyeball!! of Sheep! on Long! Neck! Slimy!! Tar!! Bubbling! Wet!"; Gorgeous Gruesome Portrait by Antonio José Manzanedo, by Seb McKinnon, by Ralph McQuarrie; Detailed Gouache Painting; Focused; Grotesque; Disturbing; Deep Colours; Textured; Meticulous 
"Sinister Cat! Huge Eyeball!! Long Neck! Slimy!! Gelatinous!!!! Viscous!"; Eldritch Gruesome Portrait by Antonio José Manzanedo, by Seb McKinnon, by Ralph McQuarrie; Detailed Gouache Painting; Focused; Grotesque; Disturbing; Deep Colours; Textured; Meticulous 
"Sinister Ornate Venetian! Mask!! on a Stick!"; Gorgeous Sinister Full Body Portrait by Antonio José Manzanedo, Seb McKinnon and Ralph McQuarrie; Delicate; Volumetric Lighting; Detailed Gouache Painting; Sharp Focus; Artstation; Grotesque Nightmare 
"Creepy! Slim!! Tall!!! Victorian! Man With Four! Arms! and a Cane!"; Gorgeous Sinister by Antonio José Manzanedo, Seb McKinnon and Ralph McQuarrie; Intricate; Rich Colours; Volumetric Lighting; Detailed Gouache Painting; Sharp Focus; Artstation; Grotesque Nightmare 
"Gaping Maw!!!!! Filled With Bloody Teeth!!"; Gorgeous Sinister; Close-Up Portrait; Intricate Elegant; Volumetric Lighting; Detailed Digital Painting; Sharp Focus; Ruan Jia; HR Giger; Alphonse Mucha; Grotesque Nightmare; Photorealistic; Deep Colours 
"Skeletal!!! Undead!! Horned!!!! Demon!!!! With Burning!! Crown!!"; Gorgeous Sinister; Close-up Portrait; Intricate Elegant; Volumetric Lighting; Detailed Digital Painting; Sharp Focus; Ruan Jia; HR Giger; Alphonse Mucha; Grotesque Nightmare; Photorealistic; Deep Colours 
"Mechanical Degenerescence" Beautiful, Sinister, Biomechanical, Hyper Detailed, Complex; Eldritch; Deep Colours; Regal; HR Giger, Tom Bagshaw, Sandra Chevrier, Roberto Ferri, Michael Kaluta, Todd McFarlane, Hideyuki Ashizawa, Zdzisław Beksiński, Erik Johansson. Cinematic, 8K, Photorealistic. Octane Render. SFX. Ray Traced Reflections 
The season of the Witch; by Greg Rutkowski, by Anato Finnstark; maximalist oil on canvas, 8K resolution, Behance HD, #film:: halloween, spooky, seductive 
"portrait of a Woman with a scary smile", by Ismail Inceoglu, Yoshitaka Amano, Takehiko Inoue, Junji Ito, Dao Trong Le, insanely detailed, 8k, digital illustration, trending on artstation, delicate, sinister, complex, deep colours, Cinematic, Dramatic, Nightlight, Sharp focus 
Half Portrait of a kawaii zombie chibi girl, creepy, sinister, scary, horror; centered, highly detailed, d&d, fantasy, highly detailed, photorealism, digital painting, trending on artstation, concept art, sharp focus, illustration; art by artgerm & greg rutkowski & fuji choko & viktoria gavrilenko & hoang lap 
Portrait of Johnny Depp, by Greg Rutkowski, Giger, Gustave Doré, Wadim Kashin, hyperdetailed Eldritch, 8k, concept art, 3D shading, 3Delight, 3ds Max, CGSociety, CryEngine, Horde3D, Octane Render, Substance Designer, shadow depth, VRay, ZBrush, hyperrealism, trending on Artstation 
Portrait of Baba Yaga, by Greg Rutkowski, Giger, Gustave Doré, Wadim Kashin, hyperdetailed Eldritch, 8k, concept art, 3D shading, 3Delight, 3ds Max, CGSociety, CryEngine, Horde3D, Octane Render, Substance Designer, shadow depth, VRay, ZBrush, hyperrealism, trending on Artstation 
HORRORCORE THIS IS SPARTA, THIS IS NIGHTMARE FUEL, 8k 3D Octane Render Meat, Skinned Screaming Blood Meaty Pit 
Highly detailed portrait of pinhead from hellraiser by alex grey, patrick woodroffe, mark ryden created by gustave dore and greg rutkowski, high detailed, smooth draw, synthwave neon retro, intricate, realistic proportions, dramatic lighting, trending on artstation 
Dream highly detailed portrait of pinhead hellraiser, magnificent, photographic realistic background, by atey ghailan, by greg rutkowski, by greg tocchini, by james gilleard, by joe fenton, by kaethe butcher, trending on instagram, award winning details 
Portrait of hellraiser cenobite in the style of wayne barlowe, gustav moreau, peter mohrbacher, bussiere, roberto ferri, santiago caruso, luis ricardo falero, austin osman spare 
Portrait of pinhead from hellraiser. demonic cenobite. oil painting by lucian freud. path traced, highly detailed, high quality, j. c. leyendecker, drew struzan tomasz alen kopera, peter mohrbacher, donato giancola 
Beautiful illustration, layout of poster, japanese style, album cover art, risograph, typography, logo, landscape, retro box cover art; Simon Stalenhag; influenced by retro & vintage, artstation, 8k, 80s box art 
"Only good dreams this time"; ultradetailed, intricate, mythpunk, beautifully colored, ink illustration; Rodney Frederickerson, Kyle Krauskopf, Angelita Martinez, Casssie Murphy, Raffaello Ossola, Rovina Cai; album cover art, book cover art, poster art, 3d liquid art, 8k, vivd, hyperrealism, a masterpiece 
A lively and whimsical dark fungal pawn shop, cinematic framing, wide angle, rain lit, kawaii shop grows from the stalk of a giant mushroom, cgsociety, siggraph, dystopian scifi, concept art, set design, oleg oprisco, conrad roset, anka zhuravleva, gediminas pranckevicius, cornell, kawasaki 
The sun shines into a city apartment, neatly cluttered, comfort, a highly intricate and hyperdetailed matte photography by Ismail Inceoglu, Sakiyama, James Gurney, Victo Ngai, Gregorio Catarino, M.W Kaluta and Hiroyuki-Mitsume Takahashi, album cover art, poster art, long exposure, cel-shaded, a masterpiece 
a teenager room in the 80's, posters on the wall, old computer on a desk, by dan witz, Wadim Kashin, junji ito, greg rutkowski, Benedetto Fellin, ultra detailed, hyperdetailed, intricate and refined, cel-shaded, deviantart, post-production; vfx, album cover art, poster art 
A lighthouse with light beam, huge waves against large rocks, glittering splash; Ismail Inceoglu, Dan Witz, Junji Ito,Kaluta, Kandinsky; fantasy art, album cover art, photorealism, ultra-detailed, 16k, digital illustration, cel-shaded, a masterpiece, nightscape, moonlit 
A lighthouse with light beam; Ismail Inceoglu, Dan Witz, Junji Ito,Kaluta, Kandinsky; fantasy art, album cover art, photorealism, ultra-detailed, 16k, digital illustration, cel-shaded, a masterpiece, nightscape, moonlit 
tornado striking farmers at work; ultradetailed, surreal, vfx, pastel hues; Stephen Bliss, Greg Rutkowski, Loish, Rhads, Beeple, Makoto Shinkai, Louis van Baarle, Ilya Kuvshinov, Rossdraws, Tom Bagshaw; unreal engine, global illumination, photorealism, 16k, detailed & realistic environment 
Spooky fairytale forest, row of witch huts in the trees; intricate, elegant, highly detailed, hyperrealistic, sharp focus, cinematic lighting, masterpiece; Rutkowski, Zabrocki, Karlkka, Jayison Devadas, James Paick, John Park, Frazetta, John Howe, Ruan Jia, Jeffrey Catherine Jones; 16k, ultra wide, zenith view, pincusion lens 
A man stands in the forefront on a hill overlooking a New orleans style nightscape, crystal lights shine in windows, shade render!, midnight colors, photograph taken by giger and beksinski, death fog and decaying megacity 
Haunted winter forest; A Beautifully Colorful And Detailed Mixed Media Ink Illustration By Ismail Inceoglu, Junji Ito, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, Album Cover Art, 3d Liquid Detailing, Subsurface Scattering, Octane Render, VRay, 16k Resolution, Vivid, Asaf Hanuka, Anna Dittmann, Hyperdetailed, hyperrealism 
Autumn Forest, by Greg Rutkowski, Giger, Gustave Doré, Wadim Kashin, hyperdetailed Eldritch, 8k, concept art, 3D shading, 3Delight, 3ds Max, CGSociety, CryEngine, Horde3D, Octane Render, Substance Designer, shadow depth, VRay, ZBrush, hyperrealism, trending on Artstation 
Beautiful houses made from junkyard scraps, in an overgrown field; architectural render, futuresynth, chillwave, scrapwave architecture, blender; artstation, Gal Barkin; post-apocalyptic post-soviet city in background; painting by Jakub Rozalski, artstation, deviantart; hyperdetailed, hyperrealistic, 4k uhd 
Matte painting of the 4 seasons on a surreal landscape, glowing rainbow bridge, enchanted dream world, seasons!!, colored flowers, mystic hues, bioluminescence; surrealism!!!; Thomas Kinkade, Lee Madgwick, Bosch, Caras Lonut, Gediminas Pranckevicius, Mimo Rotella, Alberto Seveso; ultra-detailed, foggy, 4k, acid pixie, dramatic lighting 
80s futuristic outdoor retro disco, lush vegetation; desolate, orante, dynamic, particulate, intricate, elegant, highly detailed, centered; Simon Stålenhag, Beeple, James Gilleard, Justin Gerard, Artstation; smooth, sharp focus, 3d, octane, album art cover, record jacket 
A lighthouse beam, by Ismail Inceoglu, Dan Witz, Junji Ito, M.W. Kaluta and Kandinsky, fantasy art, album cover art, photorealism hyperdetailed, 8k resolution, digital illustration, cel-shaded 
wanderer above the sea of fire Wayne Barlowe John Martin Eliran Kantor Caspar David Friedrich warm colors tetradic colors romanticism oil on canvas complex beautiful elegant firey landscape maximalist renaissance painting 
"Small Enchanted Victorian Castle Atop Narrow Mountain Spire" Hyperdetailed Digital Painting, Beautiful, Ethereal; Divine; Complex, Deep Colours by Ferdinand Knab, Raffaello Ossola, Simon Stålenhag and Erik Johansson. Photorealistic. Octane Render. SFX. Ambiant Occlusion. Ray Traced. Ethereal. Magical. Whimsical. 
Gediminas pranckevicius fish eye view of detailed portrait of floating miniature desert island with oasis planet, intricate complexity, by greg rutkowski, ross tran, conrad roset, takato yomamoto, ilya kuvshinov palm trees, small lake, birds, rocks, painting by lucian freud and mark brooks, bruce pennington, bright colors, neon, life, god, godrays, sinister hall smoke 
Natural landmarks of kagoshima, contemporary commercial art spot illustration in the style of thom wasselmann and terry pastor, minimalist airbrush art painting clean spot color, isolated on whites with fluorescent neon spray paint airbrush accents soft puffy fluffy visceral graphic design volumetric octane render iconographic minimalism 
The ancient oracle of the luminous dream spirits on cosmic cloudscape, futurism, dan mumford, victo ngai, da vinci, josan gonzalez 
Documents, papers and folders floating in the sky, a nuclear blast on the horizon, beksinski, dariusz zawadzki, very coherent symmetrical artwork. cinematic, hyper realism, high detail, octane render, 8 k 
a epic view of a mountainous lake, concept art, trending on, very detailed, unreal engine, 4 k, photoreal, volumetric lighting, light rays, epic composition 
pygmy puff, kawaii, sad fluffy monster, big eyes, unicorn horn, cartoon creature, tarantula cat hybrid, striped; digital illustration; artstation, hannah yata charlie immer, wlop, james jean; soft lighting, 16k, photorealistic, low angle, sharp focus, centered, vray, 3d render 
Kawaii fantasy monster, big soulful eyes, geometric patterned fur, holding heart-shaped balloons; Swoon, KimKeever, ToddSchorr, IsmailInceoglu, Kaluta, AntonFadeev, YoshitakaAmano; ultra-detailed, 16k, digital illustration, photorealism, trending on artstation, natural lighting, SFX, CGSociety, Vivid, bright 
Kawaii fantasy monster, big soulful eyes, geometric patterned fur, holding heart-shaped balloons; Swoon, Kim Keever, Todd Schorr, Rossdraws, Ismail Inceoglu, Kaluta, Anton Fadeev, Yoshitaka Amano; ultra-detailed, 16k, digital illustration, photorealism, natural lighting, SFX, CGSociety, Vivid, bright 
cat skeleton; A Beautifully Colorful And Detailed Mixed Media Ink Illustration By Ismail Inceoglu, Junji Ito, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, Album Cover Art, 3d Liquid Detailing, Subsurface Scattering, Octane Render, VRay, 16k Resolution, Vivid, Asaf Hanuka, Anna Dittmann, Hyperdetailed, hyperrealism 
dog skeleton hatter; A Beautifully Colorful And Detailed Mixed Media Ink Illustration By Ismail Inceoglu, Junji Ito, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, Album Cover Art, 3d Liquid Detailing, Subsurface Scattering, Octane Render, VRay, 16k Resolution, Vivid, Asaf Hanuka, Anna Dittmann, Hyperdetailed, hyperrealism 
Cute lunarpunk detective skeleton cat holding a magnifying glass, Asaf Hanuka, Anna Dittmann, storybook illustration, fantasy art, concept art, vivid, bright, colorful, 16k, octane render 
Cute Lunarpunk cat skeleton, Asaf Hanuka, Anna Dittmann, storybook illustration, ink drawing, 16k, octane render, fantasy art, concept art, hyperdetailed, hyperrealism, vivid, dark, colorful 
Masterpiece psychedelic freaky creepazoid,beautiful, hypermaximalist, unique and bespoke, brilliant horror neosurrealism by Andreas Franke and Alexander Jansson and Ted Nasmith, Sinister vibes by Ernst Haas and David Choe , stylized photorealism by Darek Zabrocki and Max Dupain, insanely detailed intricate, coherent blending, UHD 
Dream of Eldritch Lovecraftian Monster Cute Jelly Horror; Michael Cina, Alberto Seveso, Cyril Rolando, Dan Mumford; Meaningful Experimental Visual Art, Detailed Strange Painting, Digital Illustration, Unreal Engine, 16k Resolution, Photorealism, depth, glowing, ominous, moody, beautiful, strange 
A Dream of Eldritch Lovecraftian Monster Cute Jelly Horror by Michael Cina, Alberto Seveso, Cyril Rolando and Dan Mumford, Meaningful Experimental Visual Art, Detailed Strange Painting, Digital Illustration, Unreal Engine, 8k Resolution, Photorealism, lifelike, amazing depth, glowing, serene, ominous, moody beautiful strange detailed painting 
female furry mini, cute, chibi, hedgehog girl, prickly spines, black red and white clothes, heart shape anime hair; character design conteset winner, character design contest winner, character adoptable, character design by Cory Loftis, Fenghua Zhong, Ryohei Hase, Ismail Inceoglu, Ruan Jia; 3d demo reel avatar, anime girl; style by maple story, zootopia; soft lighting, SFX, CGSociety 
"Cute Group of Happy Monsters Eating Birthday Cake" by Aeron Alfrey, Bernie Wrightson, Todd McFarlane, Boris Groh, Ralph Steadman, Dan Mumford and Michael Kaluta. Beautiful Sinister Horror, Cartoonish Realistic, Deep Colours, Hyperdetailed, 8k Resolution 
"Chibi Group of Eldritch Pokémons" by Aeron Alfrey, Bernie Wrightson, Todd McFarlane, Boris Groh, Ralph Steadman, Dan Mumford and Michael Kaluta. Beautiful Sinister Horror, Cartoonish Realistic, Deep Colours, Hyperdetailed, 8k Resolution 
"portrait of a cute birthday cake creature with big eyes", by Ismail Inceoglu, M.W. Kaluta, Anton Fadeev and Yoshitaka Amano, insanely detailed, 8k resolution, digital illustration, trending on artstation, delicate 
cute little creature, made of hemp, with a head in the form of a cannabis bloom, like baby grut, green skin, in full height, character, art by james jean and greg rutkowski, realistic face, digital art, chibi style, golden ratio, perfect composition, trending on artstation, 8k 
"Cute Fluffy Cthulhu - Big Round Eyes - Cutest Thing Ever - Tiny Wings" Perfectly Center; Photorealistic, Beautiful, Ethereal, Fluffy, Hyper Detailed, Cute, Cinematic, 8K, Photorealistic. Octane Render. SFX. Ray Traced Reflections. Ambient Occlusion 
scarlett johansson, scarlet woman, unicursal hexagram tattoo on forehead, heavenly harlot, thelema, sensual seductive bare chested Aphrodite in a space with stars planets, by James Jean, Mohrbacher and Rutkowski, swirling , elegant gradients, 3d liquid detailing, textured, intricate, octane render, 8k, bare chest, lingerie nudity 
"Aztec Mask" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman" 
a gypsy hippy steampunk caravan parc during a rainy day, a highly intricate and hyperdetailed matte painting by Dan Mumford, James Gurney, Victo Ngai, Dan Witz, Kandinsky and Ismail Inceoglu, fantasy art, album cover art 
"The Suggestion Box", intricate, elegant, contoured spherical complex sculpture, slot for suggestions, post-it notes, hyperdetailed, concept, sharp focus, illustration, greg rutkowski, meticulous detailing, WLOP, Artgerm, ray tracing, DSLR, HDR, Julia Pott, Giger, Corona Render, Unity 3D, Smoothed Out 
Early colour 8k photo of the miracle of life by goro fujita and simon stalenhag, by h.r.giger, vania zouravliov, by ayami kojima, amano, takato yamamoto. by ayami kojima, amano, karol bak, greg hildebrandt, and mark brooks, neo-gothic, gothic, rich deep colors. beksinski painting, masterpiece. creepy, diabolical, dark, mystical, intrincate, maximalism, octane rendering, cinematic, hyperrealism, octane rendering, 8k, depth of field, bokeh. iridescent accents. Vibrant 
street art brickwall building, highly detailed concept art by Erik Johansson, WLOP, Albert Seveso, Alejandro Burdisio, Michael Cina, Erin Hanson, Lee Magdwick, vray, sharp focus, intricately refined textures, urban culture, stunning, a masterpiece, deviantart, trending on artstation 
street art workshop, highly detailed concept art by Erik Johansson, WLOP, Albert Seveso, Alejandro Burdisio, Michael Cina, Erin Hanson, Lee Magdwick, cityscape, vray, sharp focus, intricately refined textures, urban culture, stunning, a masterpiece, deviantart, trending on artstation, graffiti, vaporwave 
"Black and White Asian Woman! Holding Umbrella!" Gorgeous Delicate Black and White Portrait by Joshua Miels, by Anton Fadeev, by Jimmy Law; Intricate; Elegant; Volumetric Lighting; Detailed Gouache Painting; Sharp Focus 
"Fragmented Asylum - Fractured Shards of Insanity - Crumbling Mind" Beautiful; Poster Art; Pieces; Hyper Detailed; Surreal; Eldritch; Deep Colours; Cinematic; 8K; Photorealistic Suprematism; Voxel; Tom Bagshaw; Micaela Lattanzio; Sandra Chevrier; Shintaro Kago; Alex Grey; Ross Tran; Beeple; Ralph Horsley; RHADS; Michael Kaluta; Victor Adame Minguez; Todd McFarlane; Alphonse Mucha; Erik JohanssonSFX; UHD; C4D; UE4; Octane Render 
If I get tomorrow I will do it all again:: intricately detailed geometric Graffuturism by c215, 3dshading" 
Goddess of death & decay, standing in skulls, holding back life & growth; Anna Dittmann; clothes in bones & skulls, ultra detailed, maximalist matte painting; pale, dark hues of death; 16k, photorealism, HD, DSLR, polished, realistic digital painting, full portrait of a goddess 
pop surrealism, lowbrow art; realistic cute mossy girl, sultry pose, victorian steampunk clockpunk gothicpunk fashion, moss forest; hyperrealism, muted colors; rococo, swoon, edouard bisson, richard avedon, natalie shau, loreta lux, tom bagshaw, mark ryden, trevor brown; 16k, photorealism, SFX, CGSociety 
Rotting skeleton queen covered in spider swarm, crown; Sinister, pop surrealism, lowbrow art, grotesque, Hyperdetailed,, intricate, Eldritch, vivid;  rococo, swoon, edouard bisson, richard avedon, natalie shau, loreta lux, tom bagshaw, mark ryden, trevor brown; 16k, photorealism, SFX, CGSociety 
Impossibly cute magician, ornate dress & choker, magical lights; Sinister bg, pop surrealism, lowbrow art, Hyperdetailed, intricate, Eldritch,scary, vivid; rococo, swoon, EdouardBisson, RichardAvedon, NatalieShau, LoretaLux, TomBagshaw, MarkRyden, TrevorBrown; 16k, photorealism, SFX, CGSociety 
"Ice Goddess, long beautiful watery hair"; full body portrait, horizontal symmetry!, elegant, highly intricate, ultra-detailed, ornate background, digital painting, concept art, sharp focus, crispy quality, volumetric lighting; Kay Nielsen, Zeen Chin, Wadim Kashin; 8k, photorealistic, unreal engine 
Queen of Hearts; a beautifully colorful and detailed mixed media ink illustration by Ismail Inceoglu, Junji Ito, Asaf Hanuka, Anna Dittmann, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, album cover art, 3d liquid detailing, subsurface scattering, octane render, VRay, 16k resolution, hyperrealism, vivid 
PopSurrealism, lowbrow art; Cute realistic Steampunk clockpunk gothicpunk Woman, Intricate Choker,Full Frontal, Rococo, Swoon, EduardBisson, SteveHanks, LoretaLux, TomBagshaw, MarkRyden, TrevorBrown, EveVentrue; Vibrant Colors, Photorealistic, HyperDetailed, 16k, CGSociety, Digital Illustration, SFX 
Cute realistic mythpunk gothpunk woman, hat with lights clocks gears, ornate puffed sleeve dress; Asaf Hanuka, Anna Dittmann, Ngai, Swoon, Edouard Bisson, Loreta Lux, Mar Ryden, Trevor Brown Rococo; centered, 16k, photorealism, SFX, CGSociety 
PopSurrealism, lowbrow art; Cute realistic Steampunk mythpunk gothicpunk Woman, Intricate Choker,Full Frontal, Rococo, Swoon, EduardBisson, SteveHanks, LoretaLux, TomBagshaw, MarkRyden, TrevorBrown, EveVentrue; Vibrant Colors, Photorealistic, HyperDetailed, 16k, CGSociety, Digital Illustration, SFX 
PopSurrealism, lowbrow art; Cuate realistic Steampunk clockpunk gothicpunk Woman, Intricate Choker,Full Frontal, Rococo, Swoon, EduardBisson, SteveHanks, LoretaLux, TomBagshaw, MarkRyden, TrevorBrown, EveVentrue; Vibrant Colors, Photorealistic, HyperDetailed, 16k, CGSociety, Digital Illustration, SFX 
pop surrealism, lowbrow art, full portrait; realistic cute mossy girl, sultry pose, victorian steampunk fashion, hyperrealism, muted colors; rococo, natalie shau, loreta lux, tom bagshaw, mark ryden, trevor brown; 16k, photorealism, SFX, CGSociety 
grotesque white magic kawaii witch beautiful dress; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
portrait of white magic insane putin; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
pop surrealism, lowbrow art, realistic cute mossy girl, steampunk fashion, hyper realism, muted colours, rococo, natalie shau, loreta lux, tom bagshaw, mark ryden, trevor brown style 
Beautifully feminine face; spotty pointism, bright, vivid colors, bold colors, colored spots, mixed liquied acrylic, 3d liquid art, a masterpiece, hyperdetailed, hyperrealistic; trending on artstation; 16k, octane, album cover art, poster art 
32k photorealistic splash screen portrait of an Angelic woman with peacock headdress (lots of eyes)!!! & (6 wings)!!; digital painting, bright, bold, colorful, perfectly centered, extremely intricate, ornate, ultra-maximalist, extremely detailed; Chie Yoshii, Artstation, Deviantart, Anna Dittmann, Arcane style by Fortiche Studios; unreal engine, octane 
BeautifulShadowing3DShadowingReflectiveSurfaces8KBeautifullyDetailedPencilIllustration Intricate RangeMurata AlphoseMucha BoldComplimentaryColors - 4Seasons Style Portait Fine Pencil 
"Portrait of a beautiful fire princess", by Hayao Miyazaki, Ismail Inceoglu, M.W. Kaluta and Yoshitaka Amano, insanely detailed, 8k resolution, digital illustration, photorealism, trending on artstation, Vibrant Colours, delicate, backlit, close-up 
"Portrait of cute girl with flames as hair!!! burning fire!", by Hayao Miyazaki, Ismail Inceoglu, M.W. Kaluta and Yoshitaka Amano, insanely detailed, 8k resolution, digital illustration, photorealism, trending on artstation, delicate, close-up, backlit 
"portrait of a sad haunted soul of an unwanted boy", by Ismail Inceoglu, Yoshitaka Amano, Takehiko Inoue, Junji Ito, Dao Trong Le, insanely detailed, 8k, digital illustration, trending on artstation, delicate, sinister, complex, deep colours, Cinematic, Dramatic, Nightlight, Sharp focus, close-up 
"Portrait of a cute little water spirit", by Ismail Inceoglu, M.W. Kaluta, Anton Fadeev and Yoshitaka Amano, insanely detailed, 8k resolution, digital illustration, photorealism, trending on artstation, Vibrant Colours, delicate, happy 
"Arthur Morgan red background", Red Dead Redemption, deep colours, delicate, tragic, sinister, concept art, Dao Trong Le, greg rutkowski, Alphonse Mucha, Tom Bagshaw, 8k, digital illustration, photorealism, close-up, soft, watercolor, complex, trending on artstation 
Impossibly beautiful patchwork woman, made of a collage, newspaper film, gorgeous, pop art by Pär Olofsson, by Gustave Doré, Karol Bak, 35mm lens, Desaturated, High Contrast, Kodachrome, Contre-Jour, Lumen Global Illumination, Ray Tracing Reflections, insanely detailed intricate, Unreal Engine 5, pin up 
Masterpiece, psychedelic interdimensional woman, beautiful, hypermaximalist, gorgeous by Irakli Nadar unique and bespoke, style drip by John Berkey, brilliant horror neosurrealism by Andreas Franke and Jean-Paul Riopelle and Ted Nasmith, Sinister vibes by Ernst Haas and David Choe, insanely detailed intricate by Joe Fenton, coherent blending, UHD 
Masterpiece, psychedelic nightmare horrors woman, beautiful, hypermaximalist, gorgeous by Irakli Nadar unique and bespoke, style drip by John Berkey, brilliant horror neosurrealism by Andreas Franke and Jean-Paul Riopelle and Ted Nasmith, Sinister vibes by Ernst Haas and David Choe, insanely detailed intricate by Joe Fenton, coherent blending, UHD 
Goddess of the Death and Decay, standing in skulls, holding back Life & Growth, by Anna Dittmann. Clothed in bones and skulls. Incredibly detailed, maximalist matte painting. Pale, dark hues of death. 8K resolution, HD, DSLR, polished, realistic oil painting. Portrait of a Goddess 
Portrait of a beautiful african girl, flowy blonde hair, cinematic lighting, highly detailed, digital painting, trending on artstation, pixiv, concept art, sharp focus, illustration, art by ross tran and wlop 
Symmetry!! jade, machine parts embedded into face, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha, 8 k 
Tristan eaton, victo ngai, artgerm, perfect princess 
“Daydreamer” Intricate, light rainbow splash, Scenic, Hyperdetailed, Delicate, Complex, Royo, Deep Colors, Bagshaw, Chevrier, Ferri, Kaluta, Minguez, Mucha. Cinematic, WLOP, 8K, concept, sharp focus, illustration, rutkowski, detailed eyes, Artgerm, Giger, wide angle lens, Mandelbulber3d 
The Ayahuasca Spirit, by Jean Baptiste Monge, Gilles Beloeil, a masterpiece, trending on artstation 
***Second***The Mad Hatter, by Greg Rutkowski, Giger, Gustave Doré, Wadim Kashin, hyperdetailed, 8k, concept art, 3D shading, 3Delight, 3ds Max, CGSociety, CryEngine, Horde3D, Octane Render, Substance Designer, shadow depth, VRay, ZBrush, hyperrealism, deviantart, trending on Artstation 
***First***Portrait of a mad hatter, hyperdetailed Eldritch, 8k resolution, concept art, by Greg Rutkowski, H.R. Giger, Gustave Doré, Wadim Kashin, 3D shading, 3Delight, 3ds Max, CGSociety, CryEngine, Horde3D, Octane Render, Substance Designer, shadow depth, trending on Artstation deviantart VRay ZBrush 
Caterpillar smoking hooka and tripping on lsd in wonderland, intricate detail, painting, royo, frazetta, whealan 
"Allow Your Future”, Intricate, green/white/blue, Scenic, Hyperdetailed, Royo, Embrace, Bagshaw, Chevrier, Roberto Ferri, Michael Kaluta, dreamy, Cinematic, WLOP, 8K, concept, sharp focus, rutkowski, detailed emerald eyes, Artgerm, Giger, beams, Michael Cina rainbow splashing 
”nine to the universe”, Intricate, rainbow and charcoal, Scenic, Hyperdetailed; Royo, becoming reality, Bagshaw, Chevrier, Ferri, Kaluta, Minguez, understated, Mucha, Cina. Love, Cinematic, WLOP, 8K, concept, smooth sharp focus, rutkowski, detailed eyes, Artgerm, Giger, cinema 4d 
"Sanguine Dweller”, Intricate, light green orange and purple, Scenic, Hyperdetailed; Complex; Royo; Unusual Colors; Regal; Bagshaw, Chevrier, Ferri, Kaluta, Minguez, Frazzled, warped. Cinematic, WLOP, 8K, concept, smooth sharp focus, diagrammatic, rutkowski, detailed eyes, Artgerm, Giger 
“What If?”, tom bagshaw, magali villenueve, antique hyperdetailed, digital illustration, cel-shaded, photo-illustration, dark, elaborate, serenity, junji ito, disintegrating, trending on Artstation, r/Art, shadow depth, despair, 8k resolution DSLR, Kelly Freas, romanesque Ukiyo-e, Octane Render 
"Emergence”, Intricate, Emerging from iridescent pool, Hyperdetailed, Royo, Bagshaw, Chevrier, Ferri, Kaluta, iconic, glowing edges, Michael Cina. Cinematic, WLOP, 8K, concept, smooth sharp focus, rutkowski, detailed, Artgerm, Cinema 4D, Giger, Unity 3D, Octane Render 
"Emergence”, Intricate, Emerging from pool of fractal iridescence, Scenic, Hyperdetailed, Royo, Bagshaw, Chevrier, Ferri, Kaluta, Minguez, iconic, glowing edges, Mucha, Cina. Cinematic, WLOP, 8K, concept, sharp focus, rutkowski, detailed, Artgerm, Giger, Unity 3D, Octane Render 
"Ask the Axis”, Intricate, charcoal and pastel, Scenic, Hyperdetailed, Royo, becoming reality, Bagshaw, Chevrier, Ferri, Kaluta, Minguez, understated, Mucha, Cina. Love, Cinematic, WLOP, 8K, concept, smooth sharp focus, rutkowski, detailed eyes, Artgerm, Giger, cinema 4d 
“All Grown Up” Intricate, Scenic, Hyperdetailed, Delicate; Complex; Royo; Deep Colors; Bagshaw, Chevrier, Ferri, Kaluta, Minguez, surprised, Mucha. Cinematic, WLOP, 8K, concept, illustration, rutkowski, detailed eyes, Artgerm, Giger, wide angle lens, Mandelbulber3d, Afremov, rococo red splash 
"Rapmaster G" Magical, Intricately Designed, Intricate, Elegant, Digital, Scenic, Hyperdetailed, Delicate, Complex, Royo, Rutkowski, Greg Olsen; Tom Bagshaw, Sandra Chevrier, Roberto Ferri, Michael Kaluta, Victor Minguez, Jia, Mucha, Cinematic, WLOP, JJ Abrams, HDR, Michael Cina central Rainbow Geyser 
“The Gods That Interfere With Us”, Intricate, prismatic lighting, Scenic, Hyperdetailed, Delicate, Hans Zatzka, Royo, Regal, Mysterious, Chevrier, Ferri, Kaluta, Zbrush, vlaho bukovac. Cinematic, WLOP, 8K 3D, concept Art, rutkowski, detailed eyes, Artgerm, Giger, lightning, sunshine 
Bloom, broken, bountiful, burn, beautiful, blunt :: Composite art style :: (Artgerm), (Greg Rutkowski), [Georgia O'Keeffe], [Amanda Clark], [Alfred Joseph Casson], [Gainax] :: masterpiece, Poster art, Splash art, sharp focus, Fluid lines, digital illustration, Reflections, HD, cel-shaded, fractal details, Volumetric lighting 
Beautiful D&D Character Portrait, Dark Fantasy, Detailed, Realistic Face, Digital Portrait, Intricate Armor, Fiverr Dnd Character, Wlop, Stanley Artgerm Lau, Ilya Kuvshinov, Artstation, HD, Octane Render, Hyperrealism 
Epic Lucy heartfilia from the parallel world in wonderful colors, depicted by Kael Ngu, Ivan Shishkin, Kim Jung Gi, Shepard Fairey and Samantha Cheevers 16k resolution, hyperrealism maximalistic, beautiful spectacular striking textures, amazing eye-catching coloration, impressive magnificent setting, remarkable dramatic clouds, unforgettable stunning golden point, great composition 
"So cool, amazing, gorgeous, wow!", by Ralph Steadman, by Tony Diterlizzi, by Yaoy Kusama, by H.R. Giger, by Ismail Inceoglu, ultra detailed, hyperdetailed, intricate and refined, cel-shaded, trending on artstation, great composition, masterpiece, insane, vaporwave, surrealism 
Biomechanical steampunk gothicpunk hedgehog on wheels, mechanical parts, glowing eyes, in a mystical forest, gothic baroque brutalist architecture; artgerm, giger, ellen jewett, josan gonzales, giuseppe arcimboldo; ultradetailed, photorealistic, 16k, unreal engine 5, SFX, CGSociety, creepy ambiance, muted colors, softly lit; lighting by Gregory Crewdson Light 
Biomechanical hedgehog on wheels, cybernetic implants, fractal wiring, expressive, Deep Colors, Regal, Giger, Bagshaw, Sandra Chevrier, Ferri, Kaluta, Todd McFarlane, Ashizawa, Photorealistic, HDR, rococo circuitry, dressed to impress, exquisite textures, Unity 3D, Blender3D, Unreal, light silver splashing 
Biomechanical steampunk gothicpunk hedgehog on wheels, mechanical parts, glowing eyes, in a mystical forest, gothic baroque brutalist architecture; artgerm, giger, EllenJewett, JosanGonzales, GiuseppeArcimboldo; ultradetailed, photorealistic, 16k, unreal engine 5, SFX, CGSociety, creepy ambiance, muted colors, softly lit; lighting by GregoryCrewdsonLight 
Glass sci-fi dog, red eyes, begging expression; intricate, elegant, hyperdetailed, cybernetic, digital painting; artstation, concept art, biomechanical, sharp focus, illustration; greg rutkowski; fractal details in large eyes, very striking, WLOP, Artgerm, ray tracing, DSLR, HDR, unreal engine, 16k, photorealistic 
kawaii ink splashes depicting a cyborg alien, Asaf Hanuka, Anna Dittmann, ink drawing, storybook illustration, fantasy art, concept art, vivid, bright, colorful, 16k, octane render 
Quadriplegic Soldier asking to die; dark fantasy, beautiful, colorful, intricately detailed, ink illustration; Johan Everkranz, Jean Baptiste Monge, Carlos Quevedo, Edouard Bisson, Eldritch, Marc Silvestri, Marco Mazzoni; album cover art, 3d liquid art, subsurface scattering, octane, 16k, photorealism 
Full-shot Rococo & cyberpunk style portrait; neon, seductive, no nudity; Goddess wearing a white sheet, ethereal white dripping tar, glowing green lazers, pink tigers, glowing eyes, gold halo, black gears, shiny diamonds; futuristic elements, skulls, intricate, ultra-detailed; Caravaggio; octane, cinematic, hyperrealistic, 8k, 3d shading, depth of field 
Clockpunk cyborg apple; intricate abstract, intricate artwork; Tooth Wu, WLOP, Beeple, Dan Mumford, Artstation, Rutkowski; very coherent, symmetrical, cinematic, hyperrealistic, high detail, octane, 16k, iridescent accents, photorealistic, unreal engine 
"1950s retro futuristic scifi gas station scene"; muted colors, digital painting, ultra-detailed, photorealistic, extremely intricate, crispy quality; Jean Baptiste Monge; 4k, 8k, houdini algorithmic generative render 
A full page cad drawing of a man with a steampunk brain, cosmic energy wires; surreal, deconstructed, flat, impossible geometry, ink on paper, scientific, ultra-detailed labeling, measurements, contours, smooth lines, clean edges, ultra-detailed; da Vanci, Gray's Anatomy, Mohrbacher, Moebius; 8k 
 Serious Battlepunk Anime girl striding down a busy street wearing a combat jumpsuit, carrying advanced weaponry, jets fly low through the Utopian skyscrapers. Well lit city on a stormy night. Extreme detail digital 3D HDR UHR 8k vray fantasy, beautiful Cute Kawaii, Ufotable Inc. 
Impossibly beautiful cyborg cyberpunk woman, complex electronics wires and neon, gorgeous steampunk, biomechanical, gorgeous, watercolor, pop art by Pär Olofsson, by Nick Knight, by Jovana Rikalo 35mm lens, Desaturated, High Contrast, Kodachrome, Contre-Jour, Ray Tracing Reflections, insanely detailed intricate, Unreal Engine 5 
Surreal cyberpunk city, beautiful, hypermaximalist, unique and bespoke, a masterpiece, impossible surreal architecture, warm fuzzy vibes, vibes everywhere, Ray Tracing Reflections, insanely detailed intricate, surreal, highly detailed digital matte, coherent blending, smooth geometric structure, vivid color, volumetric lighting, ambient occlusion,smooth UHD 
SciFi fantasy landscape of rainforest planet with technicolor vegitation and particles by John Stephens, by Raffaello Ossola 35mm lens, Contre-Jour, Lumen Global Illumination, Ray Tracing Reflections, insanely detailed intricate, Unreal Engine 5, Octane render, coherent blending, smooth geometric structure, volumetric lighting, ambient occlusion, hdr, 16k 
Hyperrealistic portrait of a squid monster astronaut, full body portrait, well lit, intricate abstract. cyberpunk, intricate artwork, by tooth wu, wlop, beeple. in the style of jin kagetsu, james jean and wlop, highly detailed, sharp focus, intricate concept art, digital painting, ambient lighting, 4k, artstation 
Realistic portrait of beautifully crystalized and detailed portrait of a zombie woman with gasmask, tentacles, tendrils, eldritch, matte painting of cinematic movie scene red dragon, horror, created by gustave dore and greg rutkowski, high detailed, smooth draw, synthwave neon retro, intricate, realistic proportions, dramatic lighting, trending on artstation. 
Detailed portrait neon operator girl, cyberpunk futuristic neon, reflective puffy coat, decorated with traditional japanese ornaments by ismail inceoglu dragan bibin hans thoma greg rutkowski alexandros pyromallis nekro rene maritte illustrated, perfect face, fine details, realistic shaded, fine - face, pretty face 
Realistic detailed image of futuristic cyborg-robot creating a painting with acrylic paint and brushes by ayami kojima, amano, karol bak, greg hildebrandt, and mark brooks, neo-gothic, gothic, rich deep colors. beksinski painting, part by adrian ghenie and gerhard richter. art by takato yamamoto. Masterpiece 
By kyoto animation, cool girl wearing cyberpunk intricate streetwear, beautiful, detailed symmetrical close up portrait, intricate complexity, in the style of artgerm and ilya kuvshinov, cell shaded, 4 k, concept art, by wlop, krenz cushart, greg rutkowski, pixiv. cinematic dramatic atmosphere, cinematic lighting, studio quality 
highly detailed photograph of a big red heart creating the apocalypse :: pop art :: intricate and hyperdetailed painting :: Ismail Inceoglu :: Huang Guangjian :: Victo Ngai :: Roly Edwards :: Hergé :: Ed Wilmore :: Chris Blain :: Geoff McDowall :: Gediminas Pranckevicius :: CGSociety :: uplifting :: ZBrushCentral :: sophisticated :: gorgeous :: ∆ :: sharp crisp :: polished :: formidable :: primary hues :: complex :: cel-shaded :: album cover art :: a masterpiece :: 8k resolution 
"The Dog Master" intricate, elegant, hyperdetailed, cybernetic, digital painting, artstation, concept art, biomechanical, sharp focus, illustration, greg rutkowski, fractal details in large eyes, very striking, WLOP, Artgerm, ray tracing, DSLR, HDR, unreal engine, 4k, 8k 
"The Dog Master" intricate, elegant, hyperdetailed, cybernetic, digital painting, artstation, concept art, biomechanical, sharp focus, illustration, greg rutkowski, fractal details in large eyes, very striking, WLOP, Artgerm, ray tracing, DSLR, HDR 
"Cyberpunk Chicken - Futuristic Biomechanical" Hyperdetailed Delicate; Heavenly; Complex; Eldritch; Deep Colours; Tom Bagshaw, Michael Kaluta, Mike Ploog, Todd McFarlane, Beeple, HR Giger, Alphonse Mucha, Erik Johansson. Cinematic, 8K, Photorealistic. Octane Render. SFX. Ambiant Occlusion. Ray Traced 
"Mechanical Sunflower Blooming Dripping With Honey" Metal Flower - Sinister; Hyper Detailed; Complex; Eldritch; Nightmarish; Deep Colours; by Hideyuki Ashizawa, Michael Kaluta, Todd McFarlane, Artgerm; HR Giger; Peter Gric; Sandra Chevrier, Tom Bashaw; Erik Johansson. Cinematic, 8K, Photorealistic. Octane Render. SFX. Ray Traced Reflections. Ambient Reflection 
"Mechanical Flower Blooming With Blood Spatters" Metal Flower - Sinister; Hyper Detailed; Complex; Eldritch; Deep Colours; by Hideyuki Ashizawa, Michael Kaluta, Todd McFarlane, Artgerm; Sandra Chevrier, Tom Bashaw; Erik Johansson. Cinematic, 8K, Photorealistic. Octane Render. SFX. Ray Traced Reflections. Ambient Reflection 
"Mechanised Wheels Lower Body"!!, haunted doll, glowing eyes, sharp teeth!!, teethcore!; Gorgeous Sinister Contraption by Antonio José Manzanedo, Seb McKinnon and Ralph McQuarrie; Intricate; Volumetric Lighting; Detailed Gouache Painting; Sharp Focus; Artstation; Grotesque Nightmare 
Megacorporation by shaun tan, style of john kenn mortensen 
**SECOND** Biomechanical man, cybernetic implants, fractal wiring, expressive, Deep Colors, Regal, Giger, Bagshaw, Sandra Chevrier, Ferri, Michael Kaluta, Todd McFarlane, Ashizawa, Photorealistic, HDR, rococo circuitry, dressed to impress, exquisite textures, Unity 3D, Blender3D, Unreal, light silver splashing 
**FIRST** Biomechanical man, cybernetic implants, fractal wiring, expressive, Deep Colors, Regal, Giger, Bagshaw, Sandra Chevrier, Ferri, Michael Kaluta, Todd McFarlane, Ashizawa, Photorealistic, HDR, rococo circuitry, dressed to impress, exquisite textures, Unity 3D, Blender3D, Unreal, light silver splashing 
Biomechanical bunny robot by Jean Baptiste Monge 
biomechanical steampunk creature with robotic parts and big octopus head and (glowing) eyes guarding an ancient chest in a mystic forest, gothic and baroque, brutalist architecture, ultradetailed, creepy ambiance, artgerm, giger, Intricate by Ellen Jewett and Josan Gonzalez and Giuseppe Arcimboldo 
“Brave coward”, Biomechanical, cybernetic implants, fractal wiring, expressive, Deep Colors; Regal; HR Giger, Tom Bagshaw, Chevrier, Ferri, Michael Kaluta, yellow splash, Todd McFarlane, Hideyuki Ashizawa, Photorealistic, light silver streaks, rococo hair, light yellow hues, Michael Cina, approaching, sharp definition, HDR 
“Brave Sapient”, Biomechanical, cybernetic implants, fractal wiring, expressive, Deep Colors; Regal; HR Giger, Tom Bagshaw, Chevrier, Ferri, Michael Kaluta, Todd McFarlane, Hideyuki Ashizawa, Photorealistic, light silver streaks, rococo hair, light yellow hues, Michael Cina, approaching, sharp definition, HDR 
"Ubiquity Bot", Biomechanical, cybernetic implants, H Giger, Tom Bagshaw, Chevrier, Ferri, Kaluta, Todd McFarlane, Ashizawa, Photorealistic. Amazement in eyes. IMAX, 8K 3D, HDR, antialiasing, elegant, maximal lighting, Bright yellow spherical Conduit, LED Earholes, reactive 
"Did you Mention Me?", Artgerm, Jean Tinguely, Bastien Lecouffe-Deharme, H.R Giger, Beautiful Technical Biomechanical Divine Goddess, Beautiful Lighting, Photorealistic, Hyper-Detailed, Digital Illustration, Unreal Engine, 8k 3D, Unity 3D, Octane Render, Contoured, 4K 3D 
"Await me Here" Intricate, sinister, Elegant, Digital, Scenic, Hyper-Detailed, Delicate; Complex; Luis Royo, Eldritch; Deep Colours; Regal; Tom Bagshaw, Sandra Chevrier, Roberto Ferri, Michael Kaluta, Victor Adame Minguez, Ruan Jia, Alphonse Mucha, Erik Johansson. Cinematic, WLOP 
"Freed from choices”, Intricate, Scenic, Hyperdetailed, Royo, Bagshaw, Ferri, transformational, volimetric lighting, simple, open detailed eyes, Cinematic, WLOP, concept, becoming, Artgerm, Giger, Unity 3D, Mandelbulber3D, Kintsugi 
"Cat Bot" intricate, elegant, hyperdetailed, cybernetic, digital painting, artstation, concept art, biomechanical, sharp focus, illustration, greg rutkowski, anthropomorphic, fractal details in large green cat-eyes, very striking, WLOP, Artgerm, ray tracing, DSLR, HDR 
Ultradetailed female cyborg, clockpunk, warrior, green eyes, misty background full of destruction and urban decay; 8k, scifi, fantasy, moody, fierce; photorealistic, sharp focus, asymmetrical, centered, concept art, fantasy art, artstation; artgerm, Giger, Michael Welan, Alphonse Mucha, loish, wlop 
Meticulously detailed photograph of a cute hooded robot :: pop art :: Ismail Inceoglu :: James Gurney :: Victo Ngai :: Gediminas Pranckevicius :: James Jean :: Gustave Doré :: elemental art :: scary :: horror and wlop terrifying :: sophisticated :: stunning :: distinctive lighting :: ∆ :: charming :: fantastical :: particles :: airy :: epic :: complex :: primary hues :: splash screen :: a masterpiece :: 8k resolution :: CGSociety :: ZBrushCentral :: detailed background :: fluid lines :: polished :: precise 
Stunning, Alien, Symmetrical :: Glitch FX :: ASCII art :: green hues :: Chaos :: futuristic :: Matrix text effect :: Geometrically surreal :: Photorealistic :: CGI VFX by Framestore+Weta+Digital+Industrial Light & Magic+DNEG :: beautiful cinematography :: stunning :: meticulously detailed :: popart :: splash art :: splash screen:: ∆ 
a shocked steampunk scientist with smudges on face from explosion in a lab with floating luminous crystals stars by wlop and Mandy Jurgens, loish, rossdraws, Intricate, Elegant, Digital Illustration, Scenic, Hyper-Realistic, 8k resolution, hyperdetailed digital illustration, maximalist intricate, sci-fi, 3d render, ethereal mythical fantasy 
Temptation; Surreal, vivid, bright, a mixed media ink ullustration, concept art, sophisticated, complex, intricate; Caras Lonut, Heironymos Bosch, Shaun Tan, Joe Vaux, René Magritte, Erik Johansson and Simon Stålenhag; hyperdetailed, hyperrealistic, cinematic, 8k, octane, SFX, CGSociety, distinctive lighting, splash screen 
technical pen mythpunk collision of stories lore; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Full body portrait, ginormous titan Gaia composed of earth & flowers; ultra-detailed, hyperrealistic, mixed media painting, technical pen; Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Calas Lonut; album cover art, photorealism, 16k, 4d liquid detailing, octane, vivid 
Fragmented Twin Butterflies, Crumbling Shards, Broken Dreams, Gemini Butterfly; Beautiful, Surreal, HyperDetailed, Complex, Angelic, Tragic, Eldritch, DeepColours, Cinematic, 8K, Photorealistic; Caras Lonut, TomBagshaw, RyoheiHase, JunjiIto, SandraChevrier, Beeple, RalphHorsley, Kaluta, VictorAdameMinguez; Octane, SFX 
"Music escapes us until we cry a river" HyperDetailed, Highly Surreal, Ominous, Symbolic, digital painting, vivid Colors, Ethereal; Calas Lonut, Salvador Dalí, René Magritte, Erik Johansson, Simon Stålenhag; 16K photorealistic, album cover art 
Queen of Hearts; a beautifully colorful and detailed mixed media ink illustration by Ismail Inceoglu, Junji Ito, Asaf Hanuka, Anna Dittmann, James Gurney, Ngai, Catarino, Kaluta, Kandinsky, album cover art, 3d liquid detailing, subsurface scattering, octane render, VRay, 16k resolution, hyperrealism, vivid 
technical pen, centered, grotesque, strong addiction, vintage poster; a beautifully colorful & intricately detailed mixed media ink illustration; Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, & Tetsuya Ishida; album cover art, 3d liquid art, subsurface scattering, octane render, 16k, vivid, hyperrealism 
white magic alchemist table misty smoky bubbly; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
steelpunk scene with White magic containers; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
steelpunk scene with White magic; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
technical pen mythological creature; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
an intricate surreal look into the soul; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Zombiecore narwhale floating in sky with balloons; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
The darkest secret of the human soul; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution, vivid, hyperrealism 
Portrait of a man licking a LSD acid-tab & surrounded by psychadelic hallucinations; close-up, surreal; Kawase Hasui, Moebius, Caras Lonut, Edward Hopper, James Gilleard, Beksinski, Steven Outram; colorful, flat, dark surrealism, fantasy, bright, hd, 16k, unreal engine, octane, depth of field 
"Dark Demon King in Yellow", acrylic pour painting by Jackson Pollock, Ismail Inceoglu, Ngai, Kandinsky, Peter Max, Album Cover Art, 3d Liquid Detailing, 16k Resolution, Hyperdetailed, vibrant colors 
a big heart shaped world, a highly intricate and hyperdetailed matte painting by Ismail Inceoglu, Junji Ito, James Gurney, Dan Witz, M.W Kaluta and Kandinsky, fantasy art, album cover art 
The nightmares of a God, by Ismail Inceoglu, Dan Witz, Junji Ito, M.W. Kaluta and Kandinsky, fantasy art, album cover art, photorealism hyperdetailed, 8k resolution, digital illustration, cel-shaded 
A globe, a highly intricate and hyperdetailed matte painting by Ismail Inceoglu, Dan Mumford, James Gurney, Dan Witz and Kandinsky, fantasy art, album cover art 
the tetrahedron of the infinite power, a highly intricate and hyperdetailed matte painting by Ismail Inceoglu, Junji Ito, James Gurney, Dan Witz, M.W Kaluta and Kandinsky, fantasy art, album cover art 
Giant pink god with a golden animal mask on a forest clearance surrounded by animals by beksinski 
"The Divine and the Profane" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman 
"Good and Evil" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by Todd McFarlane, by Ralph Steadman 
Lost in the other side of nowhere, digital painting. dreaming latent space. casey weldon. digital render. dan mumford. surrealism. tim white. retro. bruce pennington. 
Full body portrait Absolutely massive titan Gaia composed of earth and flowers; Colossal body made of earth intricately detailed, hyperrealistic mixed media encaustic painting by Ismail Inceoglu, Luis Royo, Yusof Gajah, Khalil Ibrahim, Bosch, Salvador Dali; maximalism, photorealism, precisionism, ambient occlusion, 4d liquid detailing, subsurface scattering, octane render 
Pop Surrealism fantasy landscape with a [mountainlake0.15] and [an oak face screaming 0.75][ erupting trees from head0.6][ in background0.45] [MossLabyrinth0.5], by Zdzslaw Belsinski, Smooth,SharpFocus,Illustration,Artgerm,DonatoGiancola,JosephChristianLeyendecker,WLOP,BorisVallejo,Artgerm 
"angel of time"; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, Tetsuya Ishida and Mahmoud Farshchian, album cover art, 3d liquid art, subsurface scattering, octane, 16k, vivid, hyperrealism 
"Keeper of Time"; a beautifully colorful and intricately detailed mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Johan Egerkrans, Rovina Cai, Laurie Lipton, Asaf Hanuka, and Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane, 16k, vivid, hyperrealism 
exhausted, Imthonofu, Enzzo Barrena, Ryouta Otsuka, Hwanggyu Kim, Junji Ito, beautiful, vaporwave, ultra detailed, hyperdetailed, intricate and refined, cel-shaded, deviantart, trending on artstation, a masterpiece, poster art, surrealism 
Hellraiser David bowie Helping helps faces Celtics knot of overlapping fingers, fingers everywhere, hand, 8k, finger dreamslike magical room, hyperrealism, the labyrinth; oil painting by James Gurney 
A robot man and a tiny dog, Lighting Balance, Compositional Unity, Compositional Balance, Compositional cohesion, cohesive, cohesion, photograph, 1920s 
StarsDanceTheWaltzThroughTheHeavensPastTheMoon. IntricatelyDesigned Hyper-RealisticMixedMediaInkPainting by IlyaKuvshinov MikeWinkelmann ColorByFredinandKnab; MaximalismPhotorealismPrecisionismAmbientOcclusion3DLiquidDetailingSubsurfaceScatteringOctaneRenderVRay16kAmazingCompositionDarkShadow 
Eyeless orb monster with many ears, tentacles holding guns, d&d, high quality, trending on artlist
Holy cephalopod with a single large knowing eye with long powerful tentacles and angelic wings, highly detailed, digital painting, artstation, concept art, matte, sharp focus, illustration, dramatic, sunset, hearthstone, art by artgerm and greg rutkowski and alphonse mucha
full shot of colorful kawaii magical chibi girl wearing dress of opalite glass & tentacles; prismatic light, opalescent surface, soft lighting; giger, ruan jia, artgerm, wlop, william-adolphe bouguereau, loish, lisa frank, artstation; sharp, intricate details, photorealistic, ultradetailed, 4k, 8k
A flying beholder creature in the style of pokemon, adorable and whimsical, fantasy, elegant, digital painting, artstation, unreal engine, octane render, concept art, matte, sharp focus, vibrant colors, high contrast, illustration, art by h. r. giger
A dramatic emotional hyperrealistic pop surrealist oil panting of a sad sobbing grotesque kawaii vocaloid figurine caricature screaming with a scrunched up red face uglycrying wrinkly featured in dead space by h r giger made of dripping paint splatters in the style of bratz, 😭 🤮 💕 🎀
Misbehaving fuzzy monster, in the style of billelis and james jean and pedro conti and stanley kubrick, inspired by die antwoord, kawaii colors, photorealistic, epic, super technical, 3 d render
3d model of a brainless sinister vibrant colored fur Cthulu creature with long fur & souless eyes; alexander jansson:1; centered, psychedelic, colorful, matte background:0.9; jim henson:0.7, dave melvin:0.4; unreal engine, deviantart, artstation, octane, finalrender, concept art, hd, 8k:0.8
Urban school zombie girl in tattered clothes fanart, dark blue long hair, muted colors, matte print, pastel colors, ornate, digital art, cute smile, digital painting, fan art, elegant, pixiv, by ilya kuvshinov, by studio ghibli
3d model of a brainless sinister vibrant colored fur creature with long fur & souless eyes; alexander jansson:1; centered, psychedelic, colorful, matte background:0.9; jim henson:0.7, dave melvin:0.4; unreal engine, deviantart, artstation, octane, finalrender, concept art, hd, 8k:0.8
Realistic vaporwave airbrush painting of grotesque kawaii madballs touhou nendoroid anime figurine wax sculptures by jack davis, yukkuri shiteitte ne
Cute monsters, colorful, digital art, fantasy, magic, trending on artstation, ultra detailed, professional illustration, chalk, poster artwork by basil gogos, clean
Girl (personification of chocolate cupcake); character design, cute hats, unreal engine, ultra-detailed, smooth, digital illustration; artgerm, tooth wu, studio ghibli; sharp, artstation, ribbons, fractal swirls, fantasy bakery, makoto shinkai, global illumination, sweets, blender, maya 8k
Full body anime portrait of a cute girl round eyes long hair dressed in a school uniform inside the school horns protruding on her head, peace sign, stunning, highly detailed, anatomically correct
Depicting a young mischievous female prep school student with medium length bright blonde hair and pale skin, in an old study room, complex artistic color ink pen sketch illustration, subtle detailing, artwork by artgerm and range murata
School girl in seifuku uniform, hyperdetailed, artstation, cgsociety, golden hour 8k
Portrait of a cute anime boy with pink hair and pink wolf ears and pink wolf tail wearing stylish clothes | | highly detailed digital art painting by ruan jia, cory loftis, jeremy mann. artstation, pinterest, volumetric lighting, subsurface scattering, photorealistic, octane render, random artists
Full body character concept art of a girl next door in chibi style | | pixar - cute - fine - face, pretty face, realistic shaded perfect face, fine details by stanley artgerm lau, wlop, rossdraws, james jean, jakob eirich, andrei riabovitchev, marc simonetti, and sakimichan, trending on artstation
A cute anime girl wearing a cat hoodie holding an umbrella walking in a neon lit street in the rain, wlop, ilya kuvshinov, artgerm, krenz cushart, greg rutkowski, hiroaki samura, range murata, james jean, katsuhiro otomo, erik jones, serov, surikov, vasnetsov, repin, kramskoi
Nekopara fantastically detailed eyes modern anime style art cute vibrant detailed ears cat girl neko dress portrait shinkai makoto studio ghibli sakimichan stanley artgerm lau rossdraws james jean marc simonetti elegant highly detailed digital painting artstation pixiv
Portrait painting of a cyberpunk tiefling street doctor, sharp focus, award - winning, trending on artstation, masterpiece, highly detailed, intricate, cartoon, anime. art by merwild and ernesto irawan and rachel denton
Portrait of the japanese moon princess kaguya hime with long flowing black hair wearing an ornate kimono with intricate floral patterns, touhou character illustration by ross tran, bo chen, toni infante, rebecca oborn, michael whelan, trending on artstation cgsociety hq
Hyperrealistic portrait of a cyberpunk man in cyberspace, by guy denning, johannes itten, russ mills, hacking effects, red eyes, detailed lines, color blocking!, acrylic on canvas, insane detail, intricate, front view, symmetrical, octane, concept art, abstract, artistic, 8k, cinematic, artstation
Low angle shot of a cyberpunk elephant robot character, intricate, elegant, highly detailed, centered, digital painting, artstation, concept art, smooth, sharp focus, illustration, artgerm, tomasz alen kopera, peter mohrbacher, donato giancola, joseph christian leyendecker, wlop, boris vallejo
Soldier from apex legends, cyberpunk futuristic neon. decorated with traditional japanese ornaments by ismail inceoglu dragan bibin hans thoma greg rutkowski alexandros pyromallis nekro rene maritte illustrated, perfect face, fine details, realistic shaded, fine - face, pretty face
Painting of a creepy family dressed with burka, intercrossed humans, animal head, in a ruined temple cave. by zdzislaw beksinski, by giger, by thierry bisch, cold hue's, warm tone gradient background, concept art, animal painting, beautiful composition
dream symmetry!; stunning portrait of beautiful vietnamese girl wearing tradition cloth!!; by victo ngai, kilian eng; vibrant colours, dynamic lighting, digital art, winning award masterpiece, fantastically beautiful, illustration; inspired by beksinski, dan mumford, artstation; greg rutkowski; 8 k
Anime key visual of a little witch with big googles, delicate features finely detailed perfect face, gapmoe yandere grimdark, trending on pixiv fanbox, painted by greg rutkowski makoto shinkai takashi takeuchi studio ghibli
A person that is themed like israel, highly detailed, digital painting, artstation, concept art, sharp focus, illustration, art by greg rutkowski and alphonse mucha
Hatsune miku holding cinnamoroll from sanrio, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha and william - adolphe bouguereau
Anime key visual of a beautiful young moon spirit girl, wearing the robe of sacred mystery, intricate, moonlit lake, stunning, highly detailed, digital painting, artstation, smooth, hard focus, illustration, art by artgerm and greg rutkowski and alphonse mucha
A beautiful portrait of a pearl goddess with glittering skin, a detailed painting by greg rutkowski and raymond swanland, featured on cgsociety, fantasy art, detailed painting, artstation hd, photorealistic
Medusa; portrait, photorealistic, ultra-detailed, multimedia ink painting, digital art, Storybook Illustration, watercolor; Artstation; Cinema 4d, Made In Maya, Blender, Z Brush And Photoshop, fantasy art, kawaii, sinister, creepy, creature design, 16k, octane
Centered View Of 3d Fluffy cursed teddy bear!! Complex 3d Render Of A cursed bear with white Fur! Detailed Fur, grey Background, Centered, Octane Render, Cinema 4d, Made In Maya, Blender, Z Brush And Photoshop, Sinister Creepy Cute Hyper Detailed, Alena Aenami Colors, Horror
Scary evil creepy unusual surreal tentacle having a plethora of eyes from an alien planet; perfectly centered, rich colors, bold, colorful, beautiful, elegant, grotesque; Anna Dittmann, Rococo, Laurie Lipton, Alyssa Monks; hyper-detailed, photorealistic, 16k, unreal engine, octane
Centered View Of 3d Fluffy Eyeball with tentacles!! Complex 3d Render Of An Eyeball With Teal Fur! Detailed Fur, Yellow Background, Centered, Octane Render, Cinema 4d, Made In Maya, Blender, Z Brush And Photoshop, Sinister Creepy Cute Hyper Detailed, Alena Aenami Colors, Horror
Beautiful model; face portrait, convex, timelapse, spotlight, watercolor, world war; Hajime Sorayama; glamour shot, light blue background, yellow highlights, isotaxal, psychedelic, Soviet art, Magazine cover, album cover art, masterpiece
Young girl with elk horns; glitchy, character design, global illumination, soft shadow, stipple, underdimensional; Ask Alexandria, Warped Tour, Asaf Hanuka; birthday card mockups, retro psychedelic posters, retro box art, retro psychedelic posters, retro Japanese travel brochures, sots art, photorealistic, unreal engine, octane
Young girl; glitchy, character design, global illumination, soft shadow, stipple, underdimensional; Ask Alexandria, Warped Tour, Asaf Hanuka, Ilya Kuvshino; birthday card mockups, retro psychedelic posters, retro box art, retro psychedelic posters, sots art, photorealistic, unreal engine, octane
glitchy, emboss, art by asaf hanuka, character design, global illumination, young girl, soft shadow, stipple, underdimensional, art by ilya kuvshinov, Birthday Card Mockups, 16k, Retro Psychedelic Posters
etching, candles, rock art, big, magic circle, pixiv, still-life, creepy, bold sketch, ineffable, soft shadow, by caspar david friedrich, by peter mohrbacher
32k, HCL, world war, art by akihiko yoshida, powerful, art by Kim Tschng Yeul, dot art, etching, still-life, letters, art by jose tapiro Y baro, steampunk blueprint, detailed body
Gediminas pranckevicius; fish eye view, detailed portrait of floating miniature desert island with oasis planet, intricate complexity; greg rutkowski, ross tran, conrad roset, takato yomamoto, ilya kuvshinov; palm trees, small lake, birds, rocks, painting; lucian freud; bright colors, life
Field guide illustration painting of a gnome by john audubon and david allen sibley, detailed art
Purple skin, tiefling, smiling, beautiful detailed eyes, cute, fantasy, intricate, elegant, highly detailed, digital painting, 4k, hdr, concept art, detailed jewelry, smooth, sharp focus, illustration, by wayne reynolds
Symmetry!! portrait of floral! borderlands 3 psycho, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski, 8 k
Now i have become death, the destroyer of worlds, artwork by artgerm, zombie ghouls, rendered in high octane, art by tuomas korpi
An insane clown, highly detailed, digital art, sharp focus, trending on art station, anime art style
Scary evil creepy unusual surreal monster from an alien planet, Anna Dittmann, hyper detailed, hyperrealism, rococo, laurie lipton, alyssa monks, 16k, unreal engine, octane
pygmy puff, kawaii, sad fluffy!! monster, big eyes, cartoon creature, cthulu; digital illustration, bright colors; pixar, lisa frank, artstation, hannah yata charlie immer, wlop, james jean; soft lighting, 16k, photorealistic, low angle, sharp focus, centered, vray, 3d render
Clockpunk cyborg guava; intricate abstract, intricate artwork; Tooth Wu, WLOP, Beeple, Dan Mumford, Artstation, Rutkowski; very coherent, symmetrical, cinematic, hyperrealistic, high detail, octane, 16k, iridescent accents, photorealistic, unreal engine
A peak inside Small Cute Kawaii fairytale bedroom; storybook illustration, nightscape, cheery, deep colors, moonlight, ornate background scene; Alexander Jansson, Gris Grimley, Andy Kehoe; 4k, illustration, highly ornate, ultra-detailed, distinct lines
Icelandic Yule Lads; dark fantasy, beautiful, colorful, intricately detailed, ink illustration; Eldritch, Johan Everkranz, Jean Baptiste Monge, Carlos Quevedo, Edouard Bisson, Marc Silvestri, Marco Mazzoni; album cover art, 3d liquid art, subsurface scattering, octane, 16k, photorealism
An evil viking necromancer king sitting on a throne of skulll; justin gerard & greg rutkowski; realistic painting, digital art, very detailed, high definition; trending on artstation
Very realistic sad screaming cyborg hedgehog by beksinski r-c 5 album cover art
Harmony of neon tiny butterfly stars, mute, floating piano pieces, floating musical notes, black-haired pianist; WLOP, James Jean, Victo Ngai; rainbow overlay in the hair!!, beautifully lit, muted colors, highly detailed; fantasy art by Craig Mullins, Thomas Kinkade
Full-body roccoco & cyberpunk style portrait; neon, seductive, purple dress; goddess wearing a white silk sheet, ethereal white dripping tar, glowing green lasers, pink cheetahs, gold halo, intricate black gears, shiny diamonds; futuristic elements, skulls; intricate, ultra-detailed, hyperrealistic; Caravaggio, artstation; octane, cinematic, 8k, photorealistic, 3d shading, depth of field
Mysterious crescent moon over magical crystal temple; bright neon colors, highly detailed, cinematic; Tim White, Eyvind Earle, Philippe Druillet, Roger Dean, Lisa Frank, Aubrey Beardsley, Ernst Haeckel
Impossibly cute magician, ornate dress & choker, magical lights; Sinister bg, pop surrealism, lowbrow art, Hyperdetailed, intricate, Eldritch,scary, vivid; rococo, swoon, EdouardBisson, RichardAvedon, NatalieShau, LoretaLux, TomBagshaw, MarkRyden, TrevorBrown; 16k, photorealism, SFX, CGSociety
surreal abstract bunny skeleton; symmetrical!!!!, a beautifully colorful & intricately detailed mixed media ink illustration; Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Eldritch, Asaf Hanuka, Tetsuya Ishida; album cover art, 3d liquid art, octane, 16k, vivid, hyperrealism
"Chronically Ill" HyperDetailed, Highly Surreal, Ominous, Symbolic, digital painting, vivid Colors, Ethereal; Calas Lonut, Salvador Dalí, René Magritte, Erik Johansson, Simon Stålenhag; 16K photorealistic, album cover art
Low saturation photograph!! giant oversized rabbit robot fusion with robo-mecha-delorean chubby mech transformers, in forest jungle; cinematic focus, polaroid, vintage, neutral colors, soft lights, foggy, overcast; oleg oprisco, thomas peschak, discovery channel, victor enrich, gregory crewdson
Faded!, Military archive!!, Full portrait!!, Muscular chest!!, Rabbit head, armored combat boots, camouflage military clothes!!!, Holsters minigun scifi hulked oversize rabbit barbarian, fog, deep jungle forest, overcast!; Cinematic focus, vintage Polaroid, neutral colors, faded!!, Soft lighting; Steve Hanks, Serov Valentin, Lisa Yuskavage, Andrei Tarkovsky
surreal space oddysey vintage poster; technical pen, a beautifully colorful and intricate mixed media ink illustration by Stephen Gammell, Raffaello Ossola, Rovina Cai, Laurie Lipton, Asaf Hanuka, Tetsuya Ishida, album cover art, 3d liquid art, subsurface scattering, octane render, 16k resolution
"portrait of a man-creature, gaping maw!!!!, Filled with detailed bloody teeth!!; sinister, nightmarishly creepy, close-up Portrait, intricate, volumetric lighting, detailed gouache painting, sharp focus, photorealistic; Ruan Jia, Giger, Mucha; grotesque, 16k, deep colors, unreal engine, octane
Adorable kawaii furry fuzzy monster holding a bunch of balloons; Beautiful illustration, layout of poster, japanese style, album cover art, risograph, typography, logo, landscape, retro box cover art; Simon Stalenhag; influenced by retro & vintage, artstation, 8k, 80s box art
Beautiful illustration, layout of poster, japanese style, album cover art, risograph, typography, logo, landscape, retro box cover art; Simon Stalenhag; influenced by retro & vintage, artstation, 8k, 80s box art
Symmetry!! product render poster vivid colors divine proportion owl, ice and snow, glowing fog intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration
"""

builtin_prompts = textwrap.dedent(builtin_prompts.strip())

In [15]:
# DO NOT NEED IF RUNNING A TEXT FILE
# DEFINITELY NEED IF USING BUILT IN.  GET YOUR HUGGY TOKEN.
# IF YOU ALREADY HAVE A SAVED MODEL AND NOT ADDING NEW PROMPTS SKIP THIS SECTION AND CHOOSE LOAD EXISTING PARROT
#IN THE NEXT SECTION

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", 
                                            revision="fp16", 
                                            torch_dtype=torch.float16, 
                                            use_auth_token="HUGGY-TOKEN-HERE") 

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.63k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

# New Section

# 1. How To Train Your Parrot

Prompt Parrot will train on the prompts provided for the number of epochs specified. The more epochs, the longer training will take. But the results will be more accurate to your prompts. If you train too long the Parrot will overfit and produce your prompts exactly. Prompt Parrot handles prompt diversity pretty well. In fact, diverse prompts generally produce better results. If a word or phrase is repeated too much the parrot will fixate on it. This may be good or bad depending on how often you want the modifier to appear in your prompt generations.

Ideal `training loss`, in my experience, is in range 0.2 - 0.3. Start with 5 epoch steps. If training loss is too high lower the steps. But if it's too low, raise the steps. 

In [ ]:
#@title 1.1 Parrot Training == Mode is important - Train New if it's the first time of course, or if you are adding new prompts to the mix.  Otherwise switch to LOAD EXISTING PARROT.

#@markdown Model Settings
mode = "Load Existing Parrot" #@param["Train New Parrot", "Load Existing Parrot"]
model_name = "prompt_parrot" #@param{type:"string"}
num_train_epochs=5 #@param{type:"integer"}

#@markdown Prompts Settings
prompts_source = "Use Kyrick Prompts" #@param["Use Kyrick Prompts", "Load Text File"]
prompts_file_path = "/content/drive/MyDrive/generative/prompt_parrot/saved_model/prompt_parrot/PromptFuel01.txt" #@param{type:"string"}

#@markdown ####Legacy Training Mode
#@markdown Legacy mode is the old 2.0 training method. It generates less coherent prompts but more creative prompts. It can be fun to use! The results can get pretty crazy. `Set num_train_epochs to at least 50`
classic_choas_mode = False #@param{type:"boolean"}
num_prompt_shuffles = 4

saved_model_folder = PROJECT_PATH / "saved_model"
saved_model_path = saved_model_folder / model_name
saved_prompt_txt_path = saved_model_path / "prompts.txt"

tokenizer = AutoTokenizer.from_pretrained("distilgpt2", cache_dir=CACHE_PATH)
tokenizer.add_special_tokens(special_tokens_dict)


mode_map = {"Train New Parrot": "train_model", 
            "Load Existing Parrot": "load_model"}

prompt_map = {"Use Kyrick Prompts": "kyrick_prompts", 
              "Load Text File": "load_prompts_txt"}

mode = mode_map[mode]
prompts_source = prompt_map[prompts_source]

# load all prompts!
all_prompts = []
if mode == "train_model":
    if prompts_source == "kyrick_prompts":
        all_prompts = builtin_prompts.splitlines()
    else:
        with open(prompts_file_path) as infile:
            all_prompts = infile.read().strip().splitlines()
else:
    with open(saved_prompt_txt_path, "r") as infile:
        all_prompts = infile.read().strip().splitlines()

# if no prompts then what we do?!
if not all_prompts:
    raise UserWarning(f"Read 0 prompts from {prompts_file_path}. Please provide prompts in a line separated file called ")


# train or load model
model = None
if mode == "load_model":
    model = AutoModelForCausalLM.from_pretrained(str(saved_model_path))
    print("Model loaded from drive. Training skipped.")
else:
    # create save location
    os.makedirs(str(saved_model_path), exist_ok=True)

    # save prompts used for training
    with open(saved_prompt_txt_path, "w+") as outfile:
        for prompt in all_prompts:
            outfile.write(prompt + "\n")

    # load up the dataset
    prompts_dataset = datasets.Dataset.from_dict({"prompts": all_prompts})
    if classic_choas_mode:
        tokenized_dataset = tokenize_group(
            tokenizer, prompts_dataset, shuffles=num_prompt_shuffles
        )
        if not len(tokenized_dataset):
            # if the batch load fails due to too few prompts, fall back to line by line
            tokenized_dataset = tokenize_line_by_line(tokenizer, prompts_dataset)
    else:
        tokenized_dataset = tokenize_line_by_line(tokenizer, prompts_dataset)


    

    # train the model
    training_args = TrainingArguments(
        output_dir='./output',
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=1,
        prediction_loss_only=True,
        logging_steps=100,
        save_steps=0,
        seed=random.randint(0, 2**32-1),
    )
    model = AutoModelForCausalLM.from_pretrained("distilgpt2", cache_dir=CACHE_PATH)
    model.resize_token_embeddings(len(tokenizer))
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset,
    )

    trainer.train()

    model.save_pretrained(str(saved_model_path))


Model loaded from drive. Training skipped.


In [ ]:
#@title 1.2 Generate prompts!

#@markdown _A prompt for my prompts?!?_ **PREPOSTEROUS!!** 
#@markdown
#@markdown GPT-2 requires a prompt string to start a sentence. Best results will resemble part of your prompts. But sometimes original starting strings can have unexpected awesome results!
#@markdown
#@markdown Three prompt modes:
#@markdown * `Starting_Words`: All prompts generated with starting words selected from the start of your prompts. Configure with `num_start_words`
#@markdown * `Promptless`: Uses the start token to generate prompts. Each prompt will start with a different word (usually from your prompts. but it can be suprising!)
#@markdown * `Manual_Prompt`: Input a `manual_prompt` below!
style = "Starting_Words" #@param ["Starting_Words", "Promptless", "Manual_Prompt"]
manual_prompt="Insert Prompt Here" #@param{type:"string"}
num_start_words = 5 #@param{type:"integer"}

num_prompts_to_generate=8 #@param{type:"integer"}
#markdown Maximum and min length of the generated prompts. Will cut off mid word. This is expected behavior
max_prompt_length=50 #@param{type:"integer"}
min_prompt_length=30 #@param{type:"integer"}
#@markdown `temperature`: Default: 1.2. Turning up will inject more chaos.
temperature=1.2 #@param{type:"number"}
#@markdown `top_k`: Default 70. The number of top tokens returned by the AI. Will be randomly selected for generation.
top_k=70 #@param{type:"integer"}
#@markdown `top_p`: Default 0.9. The total percent to consider from the `top_k` returned tokens. For more information refer to [this guide!]( https://docs.cohere.ai/token-picking/)
top_p=0.9 #@param{type:"number"}


if style == "Starting_Words":
    prompt_starts = list(set([" ".join(p.split()[0:num_start_words]).replace(",", "") for p in all_prompts if len(p.split()) > 1]))
    prompt = random.choice(prompt_starts)
elif style == "Promptless":
    prompt = start_token
else:
    if not manual_prompt:
        raise UserWarning("manual_prompt must be at least 1 letter")
    prompt = manual_prompt
    
encoded_prompt = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(model.device)

output_sequences = model.generate(
    input_ids=encoded_prompt,
    max_length=max_prompt_length,
    min_length=min_prompt_length,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    do_sample=True,
    num_return_sequences=num_prompts_to_generate,
    pad_token_id=tokenizer.pad_token_id # gets rid of warning
    )

tokenized_start_token = tokenizer.encode(start_token)

prompt_overrides = ""
prompts_to_run = []
for generated_sequence in output_sequences:
    if classic_choas_mode:
        # all the tokens! All the time! 
        tokens = list(generated_sequence)
    else:
        # precision is a virtue
        tokens = []
        for i, s in enumerate(generated_sequence):
            if not classic_choas_mode and s in tokenized_start_token and i != 0:
                if len(tokens) >= min_prompt_length:
                    break
            tokens.append(s)

    text = tokenizer.decode(tokens, clean_up_tokenization_spaces=True, skip_special_tokens=True)
    text = text.strip().replace("\n", " ").replace("/", ",") # / remove slash. It causes problems in namings
    prompts_to_run.append(Prompt(base_filename="PromptParrot", prompt=text))
    print(text) # / remove slash. It causes problems in namings

"Sinful Decadence - Pink Panther" Beautiful, Delicate, Complex, Scenic, Hyperdetailed, HyperDetailed, Sinister, Eldritch; Cinematic, 8K, concept, sharp focus, concept, sharp focus, illustration
"Sinful Decadence - Pink Panther" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by
"Sinful Decadence - Pink Panther" Beautifully Mixed Media Gouache Painting by Tom Bagshaw, Sandra Chevrier, Todd McFarlane, Ralph Steadman, Roberto Ferri and Yoshitaka Amano. Intricate, UHD
"Sinful Decadence - Pink Panther" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta, by Beeple, by
"Sinful Decadence - Pink Panther" Sinister; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Colours; by Tom Bagshaw, by Michael Kaluta; by Beeple, by Todd McF
"Sinful Decadence - Pink Panther" Beautiful; Tragic; Delicate; Nightmarish; Eldritch; Detailed; Complex; Deep Co

In [ ]:
# 1.3
# OPTIONAL: copy a subset of prompts into this block quote. Edit them if you'd like. It will override all prompts and only run these!
# example:
# prompt_overrides = """\
# a beautiful painting of a lamb in a pastoral landscape
# a twisted spire on an alien landscape by John Harris and Ross Tran
# """
prompt_overrides = """\
"""

In [ ]:
#@title 1.4 Do The Run == NOT WORKING FOR THE BOB, it tosses out errors.  NameError: name 'pipe' is not defined.  i'll work on it.
prompts_ = (p.strip() for p in prompt_overrides.splitlines() if p.strip())
overrides = [Prompt(base_filename="PromptParrot", prompt=p) for p in prompts_]    

#@markdown ## Run Settings
#@markdown set seed to -1 for random seed
batch_name = "PromptParrot" #@param {type: "string"}
#@markdown batch_size is images per prompt
batch_size = 6 #@param {type: "integer"}
manual_seed = -1 #@param {type:"integer"}
steps = 50  #@param {type:"integer"}
cfg_scale = 9.0  #@param {type:"number"}
scheduler = "k_lms" #@param ['k_lms', 'ddim', 'pndm']

#@markdown set to 0 to turn off. Number of retries per NSFW result (will try with different seed)
NSFW_retries = 0 #@param {type:"integer"}

#@markdown ##Image Settings
image_ext = "png" #@param ["jpeg", "png"]
width = 512  #@param {type: "integer"}
height = 512  #@param {type: "integer"}

#@markdown turn on/off dispalay of each image
display_single_images = False #@param {type:"boolean"}

#@markdown ##Grid Settings
make_batch_grid = True #@param {type:"boolean"}
grid_cols = -1 #@param {type:"integer"}
max_images_per_grid = 6 #@param {type:"integer"}
grid_padding = 5 #@param {type:"integer"}
#@markdown color name (`black`, `white`, `pink`, etc.) or hex code in format `#00004c'`
grid_bg_color = "black" #@param {type: "string"}

batch_settings = BatchSettings(
    batch_size=batch_size,
    batch_name=batch_name,
    base_path=IMAGES_OUT,
    steps=steps,
    scheduler=scheduler,
    cfg_scale=cfg_scale,
    seed=manual_seed,
    NSFW_retry=NSFW_retries,
    display_individual_images=display_single_images,
    image_ext=image_ext,
    image_w=width,
    image_h=height,
    make_grid=make_batch_grid,
    grid_cols=grid_cols,
    grid_max_images=max_images_per_grid,
    grid_padding=grid_padding,
    grid_bg_color=grid_bg_color
)

# from diffusers import LMSDiscreteScheduler, DDIMScheduler, PNDMScheduler
# lms = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
# pipe.scheduler = lms
set_scheduler(pipe, batch_settings.scheduler)
if overrides:
    run_prompts(pipe, overrides, batch_settings)
else:
    run_prompts(pipe, prompts_to_run, batch_settings)



starting run id: 0
running: "Sinful Decadence - Pink Panther" Beautiful, Delicate, Complex, Scenic, Hyperdetailed, HyperDetailed, Sinister, Eldritch; Cinematic, 8K, concept, sharp focus, concept, sharp focus, illustration


  0%|          | 0/6 [00:00<?, ?it/s]


RuntimeError: ignored